<p align="center">
  <img src="https://media.istockphoto.com/id/1135912032/photo/respiratory-system.jpg?s=2048x2048&w=is&k=20&c=9vBNs47uX_gaY32HKTzgO4t2I-gaYIV3LA7xZAtM1CA=" style="width:100%; max-width:900px; height:180px; object-fit:cover; border-radius:10px;"/>
</p>
<div style="text-align:center;">
  <h1 style="color:Gray; display:inline-block; margin:0;; size:2.5em">Lung Life</h1><br>
  <h1 style="color:Gray; display:inline-block; margin:0;">Aplicacion movil para diagnostico temprano de cancer pulmonar</h1>
  <p>
    <b>LungLife | Machine Learning & Data Science | CRISP-DM</b><br>
    <span style="font-size:1.1em;">Análisis y predicción de la salud pulmonar utilizando datos de imágenes médicas y machine learning.</span>
  </p>
</div>

# FASE 6: Deployment (CRISP-DM)

## Propósito de la Fase de Deployment

En esta sexta y última fase del proceso CRISP-DM, desplegamos el modelo predictivo validado en un entorno de producción. El objetivo es crear una **arquitectura desacoplada y escalable** que integre el Frontend (Ionic), Backend (Node.js) y el Microservicio de ML.

### Objetivos de esta Fase

1. **Exposición del Modelo (API REST):** Crear API con FastAPI para servir predicciones
2. **Orquestación y Comunicación:** Definir flujos entre servicios con seguridad
3. **Persistencia de Predicciones:** Diseñar esquema PostgreSQL para auditoría
4. **Contenerización (Docker):** Crear entorno aislado con Docker Compose
5. **Documentación Técnica:** Documentar API con Swagger/OpenAPI

### Arquitectura del Sistema

| Componente | Tecnología | Puerto | Función |
|------------|------------|--------|---------|
| **Frontend** | Ionic/Angular | 8100 | Interfaz de usuario móvil |
| **Backend** | Node.js/Express | 3000 | Lógica de negocio y autenticación |
| **ML Service** | Python/FastAPI | 8000 | Motor de inferencia ML |
| **Database** | PostgreSQL | 5432 | Persistencia de datos |
| **Cache** | Redis | 6379 | Caché de predicciones (opcional) |

In [34]:
# =============================================================================
# CONFIGURACIÓN E IMPORTACIONES
# =============================================================================

import warnings
warnings.filterwarnings('ignore')

import sys
import os
from pathlib import Path
from datetime import datetime
import json

print("=" * 70)
print("CONFIGURACIÓN - FASE 6: DEPLOYMENT")
print("=" * 70)

# Definir rutas del proyecto
PROJECT_ROOT = Path('../..').resolve()
MODELS_DIR = Path('../models')
DEPLOYMENT_DIR = Path('../deployment')
DEPLOYMENT_DIR.mkdir(parents=True, exist_ok=True)

# Crear estructura de directorios para deployment
dirs_to_create = [
    DEPLOYMENT_DIR / 'ml_service',
    DEPLOYMENT_DIR / 'ml_service' / 'app',
    DEPLOYMENT_DIR / 'ml_service' / 'models',
    DEPLOYMENT_DIR / 'docker',
    DEPLOYMENT_DIR / 'schemas',
    DEPLOYMENT_DIR / 'scripts',
    DEPLOYMENT_DIR / 'docs',
]

for dir_path in dirs_to_create:
    dir_path.mkdir(parents=True, exist_ok=True)
    print(f"[OK] Directorio creado: {dir_path}")

print(f"\n[INFO] Estructura de deployment inicializada")
print(f"[INFO] Fecha: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

CONFIGURACIÓN - FASE 6: DEPLOYMENT
[OK] Directorio creado: ..\deployment\ml_service
[OK] Directorio creado: ..\deployment\ml_service\app
[OK] Directorio creado: ..\deployment\ml_service\models
[OK] Directorio creado: ..\deployment\docker
[OK] Directorio creado: ..\deployment\schemas
[OK] Directorio creado: ..\deployment\scripts
[OK] Directorio creado: ..\deployment\docs

[INFO] Estructura de deployment inicializada
[INFO] Fecha: 2026-01-26 01:41:01


---

## 1. Diagrama de Arquitectura de Despliegue

### 1.1 Arquitectura General del Sistema

El siguiente diagrama muestra la arquitectura de microservicios desacoplada para el MVP de LungLife:

In [35]:
# =============================================================================
# 1.2 DIAGRAMA DE ARQUITECTURA - TEXTO ASCII
# =============================================================================

architecture_diagram = """
╔══════════════════════════════════════════════════════════════════════════════╗
║                    ARQUITECTURA DE DESPLIEGUE - LUNGLIFE MVP                 ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║    ┌─────────────────────────────────────────────────────────────────────┐   ║
║    │                         CAPA DE PRESENTACIÓN                        │   ║
║    │  ┌─────────────────┐    ┌─────────────────┐    ┌─────────────────┐  │   ║
║    │  │   App Móvil     │    │   App Web       │    │   Admin Panel   │  │   ║
║    │  │   (Ionic)       │    │   (Angular)     │    │   (Angular)     │  │   ║
║    │  │   :8100         │    │   :4200         │    │   :4201         │  │   ║
║    │  └────────┬────────┘    └────────┬────────┘    └────────┬────────┘  │   ║
║    └───────────┼─────────────────────┼─────────────────────┼────────────┘   ║
║                │                     │                     │                 ║
║                └─────────────────────┼─────────────────────┘                 ║
║                                      │                                       ║
║                                      ▼                                       ║
║    ┌─────────────────────────────────────────────────────────────────────┐   ║
║    │                         API GATEWAY / BACKEND                        │   ║
║    │  ┌─────────────────────────────────────────────────────────────────┐│   ║
║    │  │                    Node.js / Express                            ││   ║
║    │  │                         :3000                                   ││   ║
║    │  │  ┌──────────┐ ┌──────────┐ ┌──────────┐ ┌──────────────────┐   ││   ║
║    │  │  │   Auth   │ │ Patients │ │Prediction│ │    Audit Log     │   ││   ║
║    │  │  │ Service  │ │ Service  │ │ Service  │ │    Service       │   ││   ║
║    │  │  └──────────┘ └──────────┘ └────┬─────┘ └──────────────────┘   ││   ║
║    │  └─────────────────────────────────┼───────────────────────────────┘│   ║
║    └────────────────────────────────────┼────────────────────────────────┘   ║
║                                         │                                    ║
║                          ┌──────────────┼──────────────┐                     ║
║                          │              │              │                     ║
║                          ▼              ▼              ▼                     ║
║    ┌─────────────────────────────────────────────────────────────────────┐   ║
║    │                         CAPA DE SERVICIOS                           │   ║
║    │  ┌─────────────────┐    ┌─────────────────┐    ┌─────────────────┐  │   ║
║    │  │   ML Service    │    │   PostgreSQL    │    │     Redis       │  │   ║
║    │  │   (FastAPI)     │    │   (Database)    │    │    (Cache)      │  │   ║
║    │  │     :8000       │    │     :5432       │    │     :6379       │  │   ║
║    │  │                 │    │                 │    │                 │  │   ║
║    │  │ ┌─────────────┐ │    │ ┌─────────────┐ │    │ Prediction      │  │   ║
║    │  │ │ Model.joblib│ │    │ │ predictions │ │    │ Cache           │  │   ║
║    │  │ │ Preprocessor│ │    │ │ audit_logs  │ │    │ Session Store   │  │   ║
║    │  │ │ Encoders    │ │    │ │ patients    │ │    │                 │  │   ║
║    │  │ └─────────────┘ │    │ └─────────────┘ │    │                 │  │   ║
║    │  └─────────────────┘    └─────────────────┘    └─────────────────┘  │   ║
║    └─────────────────────────────────────────────────────────────────────┘   ║
║                                                                              ║
╠══════════════════════════════════════════════════════════════════════════════╣
║  COMUNICACIÓN:  HTTP/REST ──▶  │  PERSISTENCIA: PostgreSQL, Redis            ║
║  AUTENTICACIÓN: JWT Bearer     │  CONTENEDORES: Docker + Docker Compose      ║
╚══════════════════════════════════════════════════════════════════════════════╝
"""

print(architecture_diagram)

# Guardar diagrama
diagram_path = DEPLOYMENT_DIR / 'docs' / 'architecture_diagram.txt'
with open(diagram_path, 'w', encoding='utf-8') as f:
    f.write(architecture_diagram)
print(f"\n[OK] Diagrama guardado en: {diagram_path}")


╔══════════════════════════════════════════════════════════════════════════════╗
║                    ARQUITECTURA DE DESPLIEGUE - LUNGLIFE MVP                 ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║    ┌─────────────────────────────────────────────────────────────────────┐   ║
║    │                         CAPA DE PRESENTACIÓN                        │   ║
║    │  ┌─────────────────┐    ┌─────────────────┐    ┌─────────────────┐  │   ║
║    │  │   App Móvil     │    │   App Web       │    │   Admin Panel   │  │   ║
║    │  │   (Ionic)       │    │   (Angular)     │    │   (Angular)     │  │   ║
║    │  │   :8100         │    │   :4200         │    │   :4201         │  │   ║
║    │  └────────┬────────┘    └────────┬────────┘    └────────┬────────┘  │   ║
║    └───────────┼─────────────────────┼─────────────────────┼────────────┘   ║
║                │          

In [36]:
# =============================================================================
# 1.3 DIAGRAMA DE FLUJO DE PREDICCIÓN
# =============================================================================

prediction_flow_diagram = """
╔══════════════════════════════════════════════════════════════════════════════╗
║                    FLUJO DE PREDICCIÓN - SECUENCIA DE LLAMADAS               ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  ┌──────────┐     ┌──────────┐     ┌──────────┐     ┌──────────┐            ║
║  │  Usuario │     │ Frontend │     │ Backend  │     │ML Service│            ║
║  │  (Médico)│     │  (Ionic) │     │ (Node.js)│     │ (FastAPI)│            ║
║  └────┬─────┘     └────┬─────┘     └────┬─────┘     └────┬─────┘            ║
║       │                │                │                │                   ║
║       │ 1. Ingresa     │                │                │                   ║
║       │    datos       │                │                │                   ║
║       │───────────────▶│                │                │                   ║
║       │                │                │                │                   ║
║       │                │ 2. POST        │                │                   ║
║       │                │ /api/predict   │                │                   ║
║       │                │ + JWT Token    │                │                   ║
║       │                │───────────────▶│                │                   ║
║       │                │                │                │                   ║
║       │                │                │ 3. Valida JWT  │                   ║
║       │                │                │ 4. Sanitiza    │                   ║
║       │                │                │    datos       │                   ║
║       │                │                │                │                   ║
║       │                │                │ 5. POST        │                   ║
║       │                │                │ /predict       │                   ║
║       │                │                │───────────────▶│                   ║
║       │                │                │                │                   ║
║       │                │                │                │ 6. Preprocesa    ║
║       │                │                │                │ 7. Predice       ║
║       │                │                │                │ 8. Calcula SHAP  ║
║       │                │                │                │                   ║
║       │                │                │ 9. Response    │                   ║
║       │                │                │◀───────────────│                   ║
║       │                │                │                │                   ║
║       │                │                │ 10. Guarda en  │                   ║
║       │                │                │     PostgreSQL │                   ║
║       │                │                │ 11. Log audit  │                   ║
║       │                │                │                │                   ║
║       │                │ 12. Response   │                │                   ║
║       │                │◀───────────────│                │                   ║
║       │                │                │                │                   ║
║       │ 13. Muestra    │                │                │                   ║
║       │     resultado  │                │                │                   ║
║       │◀───────────────│                │                │                   ║
║       │                │                │                │                   ║
╚══════════════════════════════════════════════════════════════════════════════╝
"""

print(prediction_flow_diagram)

# Guardar diagrama
flow_path = DEPLOYMENT_DIR / 'docs' / 'prediction_flow_diagram.txt'
with open(flow_path, 'w', encoding='utf-8') as f:
    f.write(prediction_flow_diagram)
print(f"\n[OK] Diagrama de flujo guardado en: {flow_path}")


╔══════════════════════════════════════════════════════════════════════════════╗
║                    FLUJO DE PREDICCIÓN - SECUENCIA DE LLAMADAS               ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  ┌──────────┐     ┌──────────┐     ┌──────────┐     ┌──────────┐            ║
║  │  Usuario │     │ Frontend │     │ Backend  │     │ML Service│            ║
║  │  (Médico)│     │  (Ionic) │     │ (Node.js)│     │ (FastAPI)│            ║
║  └────┬─────┘     └────┬─────┘     └────┬─────┘     └────┬─────┘            ║
║       │                │                │                │                   ║
║       │ 1. Ingresa     │                │                │                   ║
║       │    datos       │                │                │                   ║
║       │───────────────▶│                │                │                   ║
║       │                │     

---

## 2. Exposición del Modelo (API REST con FastAPI)

### 2.1 Estructura del Microservicio ML

El microservicio de ML expone el modelo entrenado mediante una API REST robusta y documentada.

In [37]:
# =============================================================================
# 2.2 ESTRUCTURA DEL MICROSERVICIO ML
# =============================================================================

ml_service_structure = """
╔══════════════════════════════════════════════════════════════════════════════╗
║                    ESTRUCTURA DEL MICROSERVICIO ML                           ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  ml_service/                                                                 ║
║  ├── app/                                                                    ║
║  │   ├── __init__.py              # Inicialización del paquete              ║
║  │   ├── main.py                  # Punto de entrada FastAPI                ║
║  │   ├── config.py                # Configuración y variables de entorno    ║
║  │   ├── dependencies.py          # Inyección de dependencias               ║
║  │   │                                                                       ║
║  │   ├── api/                                                                ║
║  │   │   ├── __init__.py                                                     ║
║  │   │   ├── routes/                                                         ║
║  │   │   │   ├── __init__.py                                                 ║
║  │   │   │   ├── health.py        # Endpoints de health check               ║
║  │   │   │   └── prediction.py    # Endpoints de predicción                 ║
║  │   │   └── middleware/                                                     ║
║  │   │       ├── __init__.py                                                 ║
║  │   │       └── logging.py       # Middleware de logging                   ║
║  │   │                                                                       ║
║  │   ├── core/                                                               ║
║  │   │   ├── __init__.py                                                     ║
║  │   │   ├── model_loader.py      # Carga del modelo ML                     ║
║  │   │   ├── predictor.py         # Lógica de predicción                    ║
║  │   │   └── preprocessor.py      # Preprocesamiento de datos               ║
║  │   │                                                                       ║
║  │   ├── schemas/                                                            ║
║  │   │   ├── __init__.py                                                     ║
║  │   │   ├── request.py           # Schemas de request (Pydantic)           ║
║  │   │   └── response.py          # Schemas de response (Pydantic)          ║
║  │   │                                                                       ║
║  │   └── utils/                                                              ║
║  │       ├── __init__.py                                                     ║
║  │       ├── exceptions.py        # Excepciones personalizadas              ║
║  │       └── validators.py        # Validadores de datos                    ║
║  │                                                                           ║
║  ├── models/                       # Artefactos del modelo                   ║
║  │   ├── lung_cancer_classifier.joblib                                       ║
║  │   ├── label_encoders.joblib                                               ║
║  │   └── model_config.json                                                   ║
║  │                                                                           ║
║  ├── tests/                        # Tests unitarios e integración           ║
║  │   ├── __init__.py                                                         ║
║  │   ├── test_prediction.py                                                  ║
║  │   └── test_health.py                                                      ║
║  │                                                                           ║
║  ├── Dockerfile                    # Imagen Docker del servicio              ║
║  ├── requirements.txt              # Dependencias Python                     ║
║  └── .env.example                  # Variables de entorno ejemplo            ║
║                                                                              ║
╚══════════════════════════════════════════════════════════════════════════════╝
"""

print(ml_service_structure)

# Guardar estructura
structure_path = DEPLOYMENT_DIR / 'docs' / 'ml_service_structure.txt'
with open(structure_path, 'w', encoding='utf-8') as f:
    f.write(ml_service_structure)
print(f"\n[OK] Estructura guardada en: {structure_path}")


╔══════════════════════════════════════════════════════════════════════════════╗
║                    ESTRUCTURA DEL MICROSERVICIO ML                           ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  ml_service/                                                                 ║
║  ├── app/                                                                    ║
║  │   ├── __init__.py              # Inicialización del paquete              ║
║  │   ├── main.py                  # Punto de entrada FastAPI                ║
║  │   ├── config.py                # Configuración y variables de entorno    ║
║  │   ├── dependencies.py          # Inyección de dependencias               ║
║  │   │                                                                       ║
║  │   ├── api/                                                                ║
║  │   │   ├── __init__.py     

In [38]:
# =============================================================================
# 2.3 CÓDIGO PRINCIPAL DE FASTAPI (main.py)
# =============================================================================

fastapi_main_code = '''"""
LungLife ML Service - FastAPI Application
Motor de inferencia para predicción de cáncer pulmonar.
"""

from fastapi import FastAPI, HTTPException, Depends, Request
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse
from contextlib import asynccontextmanager
import logging
import time
from typing import Optional

from app.config import settings
from app.core.model_loader import ModelLoader
from app.api.routes import health, prediction

# Configuración de logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

# Instancia global del modelo
model_loader: Optional[ModelLoader] = None


@asynccontextmanager
async def lifespan(app: FastAPI):
    """
    Gestión del ciclo de vida de la aplicación.
    Carga el modelo al iniciar y lo libera al cerrar.
    """
    global model_loader
    
    logger.info("Iniciando ML Service...")
    logger.info(f"Cargando modelo desde: {settings.MODEL_PATH}")
    
    try:
        model_loader = ModelLoader(
            model_path=settings.MODEL_PATH,
            encoders_path=settings.ENCODERS_PATH,
            config_path=settings.CONFIG_PATH
        )
        model_loader.load()
        logger.info("Modelo cargado exitosamente")
        
    except Exception as e:
        logger.error(f"Error al cargar el modelo: {e}")
        raise RuntimeError(f"No se pudo cargar el modelo: {e}")
    
    yield
    
    logger.info("Cerrando ML Service...")
    model_loader = None


def create_application() -> FastAPI:
    """
    Factory function para crear la aplicación FastAPI.
    Sigue el principio de Clean Architecture.
    """
    application = FastAPI(
        title="LungLife ML Service",
        description="API REST para predicción de estadio de cáncer pulmonar",
        version="1.0.0",
        docs_url="/docs",
        redoc_url="/redoc",
        openapi_url="/openapi.json",
        lifespan=lifespan
    )
    
    # Configurar CORS
    application.add_middleware(
        CORSMiddleware,
        allow_origins=settings.ALLOWED_ORIGINS,
        allow_credentials=True,
        allow_methods=["GET", "POST", "OPTIONS"],
        allow_headers=["*"],
    )
    
    # Registrar rutas
    application.include_router(
        health.router, 
        prefix="/api/v1", 
        tags=["Health"]
    )
    application.include_router(
        prediction.router, 
        prefix="/api/v1", 
        tags=["Prediction"]
    )
    
    return application


# Middleware de logging de requests
@app.middleware("http")
async def log_requests(request: Request, call_next):
    """Middleware para logging de todas las requests."""
    start_time = time.time()
    
    response = await call_next(request)
    
    process_time = time.time() - start_time
    logger.info(
        f"{request.method} {request.url.path} - "
        f"Status: {response.status_code} - "
        f"Time: {process_time:.3f}s"
    )
    
    response.headers["X-Process-Time"] = str(process_time)
    return response


# Crear instancia de la aplicación
app = create_application()


def get_model_loader() -> ModelLoader:
    """Dependency injection para obtener el modelo."""
    if model_loader is None:
        raise HTTPException(
            status_code=503,
            detail="El modelo no está disponible"
        )
    return model_loader


@app.get("/")
async def root():
    """Endpoint raíz con información del servicio."""
    return {
        "service": "LungLife ML Service",
        "version": "1.0.0",
        "status": "running",
        "docs": "/docs"
    }
'''

print("=" * 70)
print("CÓDIGO PRINCIPAL DE FASTAPI (main.py)")
print("=" * 70)
print(fastapi_main_code)

# Guardar código
main_py_path = DEPLOYMENT_DIR / 'ml_service' / 'app' / 'main.py'
with open(main_py_path, 'w', encoding='utf-8') as f:
    f.write(fastapi_main_code)
print(f"\n[OK] Archivo guardado en: {main_py_path}")

CÓDIGO PRINCIPAL DE FASTAPI (main.py)
"""
LungLife ML Service - FastAPI Application
Motor de inferencia para predicción de cáncer pulmonar.
"""

from fastapi import FastAPI, HTTPException, Depends, Request
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse
from contextlib import asynccontextmanager
import logging
import time
from typing import Optional

from app.config import settings
from app.core.model_loader import ModelLoader
from app.api.routes import health, prediction

# Configuración de logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

# Instancia global del modelo
model_loader: Optional[ModelLoader] = None


@asynccontextmanager
async def lifespan(app: FastAPI):
    """
    Gestión del ciclo de vida de la aplicación.
    Carga el modelo al iniciar y lo libera al cerrar.
    """
    global model_loader

    logger.info("Iniciand

In [39]:
# =============================================================================
# 2.4 SCHEMAS DE REQUEST/RESPONSE (Pydantic)
# =============================================================================

schemas_code = '''"""
LungLife ML Service - Pydantic Schemas
Definición de schemas para validación de datos de entrada y salida.
"""

from pydantic import BaseModel, Field, validator
from typing import List, Optional, Dict, Any
from enum import Enum
from datetime import datetime


class GenderEnum(str, Enum):
    """Enum para género del paciente."""
    MALE = "Male"
    FEMALE = "Female"


class SmokingHistoryEnum(str, Enum):
    """Enum para historial de fumador."""
    NEVER = "Never"
    FORMER = "Former"
    CURRENT = "Current"


class RiskLevelEnum(str, Enum):
    """Enum para nivel de riesgo."""
    LOW = "low"
    MEDIUM = "medium"
    HIGH = "high"


class PatientDataRequest(BaseModel):
    """
    Schema de entrada para datos del paciente.
    Todos los campos requeridos para la predicción.
    """
    # Datos demográficos
    age: int = Field(..., ge=18, le=120, description="Edad del paciente")
    gender: GenderEnum = Field(..., description="Género del paciente")
    
    # Historial de tabaquismo
    smoking_history: SmokingHistoryEnum = Field(..., description="Historial de fumador")
    years_smoked: float = Field(default=0, ge=0, description="Años fumando")
    pack_years: float = Field(default=0, ge=0, description="Pack-years de tabaquismo")
    
    # Datos clínicos
    bmi: float = Field(..., ge=10, le=60, description="Índice de masa corporal")
    lung_function_test_result: float = Field(..., ge=0, le=150, description="Resultado de espirometría (%)")
    
    # Factores de riesgo
    family_history_cancer: bool = Field(default=False, description="Historial familiar de cáncer")
    exposure_to_toxins: bool = Field(default=False, description="Exposición a toxinas")
    air_quality_index: float = Field(default=50, ge=0, le=500, description="Índice de calidad del aire")
    
    # Síntomas
    chest_pain_symptoms: bool = Field(default=False, description="Dolor en el pecho")
    shortness_of_breath: bool = Field(default=False, description="Dificultad para respirar")
    chronic_cough: bool = Field(default=False, description="Tos crónica")
    weight_loss: bool = Field(default=False, description="Pérdida de peso")
    
    # Datos opcionales
    tumor_size_cm: Optional[float] = Field(default=None, ge=0, description="Tamaño del tumor en cm")
    occupation: Optional[str] = Field(default=None, description="Ocupación")
    residential_area: Optional[str] = Field(default=None, description="Área residencial")
    
    class Config:
        json_schema_extra = {
            "example": {
                "age": 55,
                "gender": "Male",
                "smoking_history": "Former",
                "years_smoked": 20,
                "pack_years": 30,
                "bmi": 26.5,
                "lung_function_test_result": 75.0,
                "family_history_cancer": True,
                "exposure_to_toxins": False,
                "air_quality_index": 85,
                "chest_pain_symptoms": True,
                "shortness_of_breath": True,
                "chronic_cough": True,
                "weight_loss": False,
                "tumor_size_cm": 2.5,
                "occupation": "Mining",
                "residential_area": "Urban"
            }
        }
    
    @validator("years_smoked", "pack_years", pre=True, always=True)
    def validate_smoking_data(cls, value, values):
        """Validar que datos de tabaquismo sean coherentes."""
        if values.get("smoking_history") == SmokingHistoryEnum.NEVER:
            return 0
        return value


class TopFeatureContribution(BaseModel):
    """Contribución de una feature a la predicción."""
    feature: str = Field(..., description="Nombre de la feature")
    value: Any = Field(..., description="Valor de la feature")
    contribution: float = Field(..., description="Contribución SHAP")
    direction: str = Field(..., description="Dirección del impacto (positive/negative)")


class PredictionResponse(BaseModel):
    """
    Schema de respuesta con la predicción y metadatos.
    """
    # Predicción principal
    prediction: str = Field(..., description="Predicción: Early o Advanced")
    prediction_code: int = Field(..., description="Código numérico: 0=Early, 1=Advanced")
    probability: float = Field(..., ge=0, le=1, description="Probabilidad de cáncer avanzado")
    confidence: float = Field(..., ge=0, le=1, description="Confianza de la predicción")
    
    # Clasificación de riesgo
    risk_level: RiskLevelEnum = Field(..., description="Nivel de riesgo: low, medium, high")
    requires_review: bool = Field(..., description="Requiere revisión médica adicional")
    
    # Interpretabilidad (XAI)
    top_factors: List[TopFeatureContribution] = Field(
        default=[], 
        description="Top factores que contribuyen a la predicción"
    )
    
    # Metadatos
    model_version: str = Field(..., description="Versión del modelo utilizado")
    prediction_id: str = Field(..., description="ID único de la predicción")
    timestamp: datetime = Field(default_factory=datetime.utcnow, description="Timestamp de la predicción")
    processing_time_ms: float = Field(..., description="Tiempo de procesamiento en ms")
    
    class Config:
        json_schema_extra = {
            "example": {
                "prediction": "Advanced",
                "prediction_code": 1,
                "probability": 0.78,
                "confidence": 0.56,
                "risk_level": "high",
                "requires_review": False,
                "top_factors": [
                    {"feature": "pack_years", "value": 30, "contribution": 0.15, "direction": "positive"},
                    {"feature": "lung_function_test_result", "value": 75, "contribution": 0.12, "direction": "positive"}
                ],
                "model_version": "1.0.0",
                "prediction_id": "pred_abc123",
                "timestamp": "2026-01-26T10:30:00Z",
                "processing_time_ms": 45.2
            }
        }


class HealthResponse(BaseModel):
    """Schema de respuesta para health check."""
    status: str = Field(..., description="Estado del servicio")
    model_loaded: bool = Field(..., description="Si el modelo está cargado")
    model_version: str = Field(..., description="Versión del modelo")
    uptime_seconds: float = Field(..., description="Tiempo de actividad en segundos")


class ErrorResponse(BaseModel):
    """Schema para respuestas de error."""
    error: str = Field(..., description="Tipo de error")
    message: str = Field(..., description="Mensaje descriptivo")
    details: Optional[Dict[str, Any]] = Field(default=None, description="Detalles adicionales")
    timestamp: datetime = Field(default_factory=datetime.utcnow)
'''

print("=" * 70)
print("SCHEMAS DE REQUEST/RESPONSE (Pydantic)")
print("=" * 70)
print(schemas_code[:2000] + "\n... [código continúa]")

# Guardar schemas
schemas_path = DEPLOYMENT_DIR / 'ml_service' / 'app' / 'schemas.py'
with open(schemas_path, 'w', encoding='utf-8') as f:
    f.write(schemas_code)
print(f"\n[OK] Schemas guardados en: {schemas_path}")

SCHEMAS DE REQUEST/RESPONSE (Pydantic)
"""
LungLife ML Service - Pydantic Schemas
Definición de schemas para validación de datos de entrada y salida.
"""

from pydantic import BaseModel, Field, validator
from typing import List, Optional, Dict, Any
from enum import Enum
from datetime import datetime


class GenderEnum(str, Enum):
    """Enum para género del paciente."""
    MALE = "Male"
    FEMALE = "Female"


class SmokingHistoryEnum(str, Enum):
    """Enum para historial de fumador."""
    NEVER = "Never"
    FORMER = "Former"
    CURRENT = "Current"


class RiskLevelEnum(str, Enum):
    """Enum para nivel de riesgo."""
    LOW = "low"
    MEDIUM = "medium"
    HIGH = "high"


class PatientDataRequest(BaseModel):
    """
    Schema de entrada para datos del paciente.
    Todos los campos requeridos para la predicción.
    """
    # Datos demográficos
    age: int = Field(..., ge=18, le=120, description="Edad del paciente")
    gender: GenderEnum = Field(..., description="Género del 

In [40]:
# =============================================================================
# 2.5 ENDPOINT DE PREDICCIÓN
# =============================================================================

predict_endpoint_code = '''"""
LungLife ML Service - Prediction Endpoint
Endpoint principal para realizar predicciones.
"""

from fastapi import APIRouter, Depends, HTTPException, BackgroundTasks
from fastapi.responses import JSONResponse
import logging
import time
import uuid
from typing import Optional

from app.schemas import (
    PatientDataRequest, 
    PredictionResponse, 
    TopFeatureContribution,
    RiskLevelEnum,
    ErrorResponse
)
from app.core.predictor import Predictor
from app.dependencies import get_predictor

router = APIRouter()
logger = logging.getLogger(__name__)


def classify_risk_level(probability: float) -> RiskLevelEnum:
    """
    Clasifica el nivel de riesgo basado en la probabilidad.
    
    Args:
        probability: Probabilidad de cáncer avanzado (0-1)
        
    Returns:
        Nivel de riesgo (low, medium, high)
    """
    if probability >= 0.7:
        return RiskLevelEnum.HIGH
    elif probability >= 0.4:
        return RiskLevelEnum.MEDIUM
    return RiskLevelEnum.LOW


def requires_medical_review(probability: float) -> bool:
    """
    Determina si la predicción requiere revisión médica adicional.
    Casos en zona de incertidumbre (0.3-0.7) requieren revisión.
    """
    return 0.3 <= probability <= 0.7


def calculate_confidence(probability: float) -> float:
    """
    Calcula la confianza de la predicción.
    Mayor distancia del umbral 0.5 = mayor confianza.
    """
    return abs(probability - 0.5) * 2


@router.post(
    "/predict",
    response_model=PredictionResponse,
    responses={
        400: {"model": ErrorResponse, "description": "Datos de entrada inválidos"},
        500: {"model": ErrorResponse, "description": "Error interno del servidor"},
        503: {"model": ErrorResponse, "description": "Modelo no disponible"}
    },
    summary="Realizar predicción de cáncer pulmonar",
    description="Recibe datos del paciente y retorna la predicción con interpretabilidad"
)
async def predict(
    patient_data: PatientDataRequest,
    background_tasks: BackgroundTasks,
    predictor: Predictor = Depends(get_predictor)
) -> PredictionResponse:
    """
    Endpoint principal de predicción.
    
    Proceso:
    1. Valida los datos de entrada (Pydantic)
    2. Preprocesa los datos
    3. Realiza la predicción
    4. Calcula explicaciones SHAP
    5. Clasifica nivel de riesgo
    6. Retorna respuesta estructurada
    """
    start_time = time.time()
    prediction_id = f"pred_{uuid.uuid4().hex[:12]}"
    
    try:
        logger.info(f"[{prediction_id}] Iniciando predicción")
        
        # Convertir request a diccionario
        patient_dict = patient_data.model_dump()
        
        # Realizar predicción
        result = predictor.predict(patient_dict)
        
        probability = result["probability"]
        prediction_label = "Advanced" if probability >= predictor.threshold else "Early"
        prediction_code = 1 if probability >= predictor.threshold else 0
        
        # Obtener factores contribuyentes (SHAP)
        top_factors = []
        if "shap_values" in result:
            for feature, shap_value, original_value in result["shap_values"][:5]:
                top_factors.append(TopFeatureContribution(
                    feature=feature,
                    value=original_value,
                    contribution=abs(shap_value),
                    direction="positive" if shap_value > 0 else "negative"
                ))
        
        # Calcular métricas adicionales
        risk_level = classify_risk_level(probability)
        needs_review = requires_medical_review(probability)
        confidence = calculate_confidence(probability)
        
        processing_time = (time.time() - start_time) * 1000
        
        response = PredictionResponse(
            prediction=prediction_label,
            prediction_code=prediction_code,
            probability=round(probability, 4),
            confidence=round(confidence, 4),
            risk_level=risk_level,
            requires_review=needs_review,
            top_factors=top_factors,
            model_version=predictor.model_version,
            prediction_id=prediction_id,
            processing_time_ms=round(processing_time, 2)
        )
        
        logger.info(
            f"[{prediction_id}] Predicción completada: "
            f"{prediction_label} (prob={probability:.4f}) "
            f"en {processing_time:.2f}ms"
        )
        
        return response
        
    except ValueError as e:
        logger.error(f"[{prediction_id}] Error de validación: {e}")
        raise HTTPException(
            status_code=400,
            detail={"error": "validation_error", "message": str(e)}
        )
    except Exception as e:
        logger.error(f"[{prediction_id}] Error interno: {e}")
        raise HTTPException(
            status_code=500,
            detail={"error": "prediction_error", "message": "Error al procesar la predicción"}
        )


@router.post(
    "/predict/batch",
    summary="Predicción en lote",
    description="Realiza predicciones para múltiples pacientes"
)
async def predict_batch(
    patients: list[PatientDataRequest],
    predictor: Predictor = Depends(get_predictor)
):
    """Endpoint para predicciones en lote."""
    if len(patients) > 100:
        raise HTTPException(
            status_code=400,
            detail="Máximo 100 pacientes por solicitud"
        )
    
    results = []
    for patient in patients:
        try:
            result = await predict(patient, BackgroundTasks(), predictor)
            results.append({"status": "success", "result": result})
        except HTTPException as e:
            results.append({"status": "error", "error": e.detail})
    
    return {"predictions": results, "total": len(results)}
'''

print("=" * 70)
print("ENDPOINT DE PREDICCIÓN")
print("=" * 70)
print(predict_endpoint_code[:2500] + "\n... [código continúa]")

# Guardar endpoint
endpoints_path = DEPLOYMENT_DIR / 'ml_service' / 'app' / 'prediction_router.py'
with open(endpoints_path, 'w', encoding='utf-8') as f:
    f.write(predict_endpoint_code)
print(f"\n[OK] Endpoint guardado en: {endpoints_path}")

ENDPOINT DE PREDICCIÓN
"""
LungLife ML Service - Prediction Endpoint
Endpoint principal para realizar predicciones.
"""

from fastapi import APIRouter, Depends, HTTPException, BackgroundTasks
from fastapi.responses import JSONResponse
import logging
import time
import uuid
from typing import Optional

from app.schemas import (
    PatientDataRequest, 
    PredictionResponse, 
    TopFeatureContribution,
    RiskLevelEnum,
    ErrorResponse
)
from app.core.predictor import Predictor
from app.dependencies import get_predictor

router = APIRouter()
logger = logging.getLogger(__name__)


def classify_risk_level(probability: float) -> RiskLevelEnum:
    """
    Clasifica el nivel de riesgo basado en la probabilidad.

    Args:
        probability: Probabilidad de cáncer avanzado (0-1)

    Returns:
        Nivel de riesgo (low, medium, high)
    """
    if probability >= 0.7:
        return RiskLevelEnum.HIGH
    elif probability >= 0.4:
        return RiskLevelEnum.MEDIUM
    return RiskLe

---

## 3. Orquestación y Comunicación

### 3.1 Diagrama de Comunicación entre Servicios

El flujo de comunicación sigue un patrón request-response con manejo de seguridad y timeouts.

In [41]:
# =============================================================================
# 3.2 DIAGRAMA DE COMUNICACIÓN ENTRE SERVICIOS
# =============================================================================

communication_diagram = """
╔══════════════════════════════════════════════════════════════════════════════╗
║                    COMUNICACIÓN ENTRE SERVICIOS                              ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  ┌─────────────────────────────────────────────────────────────────────────┐ ║
║  │                      CABECERAS DE SEGURIDAD                             │ ║
║  ├─────────────────────────────────────────────────────────────────────────┤ ║
║  │  Authorization: Bearer <JWT_TOKEN>                                      │ ║
║  │  X-Request-ID: <UUID>                                                   │ ║
║  │  X-Client-Version: <APP_VERSION>                                        │ ║
║  │  Content-Type: application/json                                         │ ║
║  │  Accept: application/json                                               │ ║
║  └─────────────────────────────────────────────────────────────────────────┘ ║
║                                                                              ║
║  ┌─────────────────────────────────────────────────────────────────────────┐ ║
║  │                      TIMEOUTS Y REINTENTOS                              │ ║
║  ├─────────────────────────────────────────────────────────────────────────┤ ║
║  │                                                                         │ ║
║  │  Frontend -> Backend:                                                   │ ║
║  │    - Timeout: 30 segundos                                               │ ║
║  │    - Reintentos: 3 (con backoff exponencial)                            │ ║
║  │    - Retry delay: 1s, 2s, 4s                                            │ ║
║  │                                                                         │ ║
║  │  Backend -> ML Service:                                                 │ ║
║  │    - Timeout: 10 segundos (predicción rápida)                           │ ║
║  │    - Reintentos: 2                                                      │ ║
║  │    - Circuit breaker: Activar tras 5 fallos consecutivos                │ ║
║  │                                                                         │ ║
║  │  Backend -> PostgreSQL:                                                 │ ║
║  │    - Timeout: 5 segundos                                                │ ║
║  │    - Pool size: 20 conexiones                                           │ ║
║  │    - Idle timeout: 10 minutos                                           │ ║
║  │                                                                         │ ║
║  └─────────────────────────────────────────────────────────────────────────┘ ║
║                                                                              ║
║  ┌─────────────────────────────────────────────────────────────────────────┐ ║
║  │                      MANEJO DE ERRORES                                  │ ║
║  ├─────────────────────────────────────────────────────────────────────────┤ ║
║  │                                                                         │ ║
║  │  Código   │ Descripción              │ Acción del Cliente              │ ║
║  │  ─────────┼──────────────────────────┼────────────────────────────────│ ║
║  │  200      │ OK                       │ Procesar respuesta             │ ║
║  │  400      │ Bad Request              │ Mostrar error de validación    │ ║
║  │  401      │ Unauthorized             │ Redirigir a login              │ ║
║  │  403      │ Forbidden                │ Mostrar acceso denegado        │ ║
║  │  408      │ Request Timeout          │ Reintentar con backoff         │ ║
║  │  429      │ Too Many Requests        │ Esperar y reintentar           │ ║
║  │  500      │ Internal Server Error    │ Mostrar error genérico         │ ║
║  │  503      │ Service Unavailable      │ Modo offline / reintentar      │ ║
║  │                                                                         │ ║
║  └─────────────────────────────────────────────────────────────────────────┘ ║
║                                                                              ║
╚══════════════════════════════════════════════════════════════════════════════╝
"""

print(communication_diagram)

# Guardar diagrama
comm_path = DEPLOYMENT_DIR / 'docs' / 'communication_diagram.txt'
with open(comm_path, 'w', encoding='utf-8') as f:
    f.write(communication_diagram)
print(f"\n[OK] Diagrama guardado en: {comm_path}")


╔══════════════════════════════════════════════════════════════════════════════╗
║                    COMUNICACIÓN ENTRE SERVICIOS                              ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  ┌─────────────────────────────────────────────────────────────────────────┐ ║
║  │                      CABECERAS DE SEGURIDAD                             │ ║
║  ├─────────────────────────────────────────────────────────────────────────┤ ║
║  │  Authorization: Bearer <JWT_TOKEN>                                      │ ║
║  │  X-Request-ID: <UUID>                                                   │ ║
║  │  X-Client-Version: <APP_VERSION>                                        │ ║
║  │  Content-Type: application/json                                         │ ║
║  │  Accept: application/json                                               │ ║
║  └───────────────────────

In [42]:
# =============================================================================
# 3.3 CLIENTE ML SERVICE PARA BACKEND NODE.JS
# =============================================================================

ml_service_client_code = '''/**
 * LungLife Backend - ML Service Client
 * Cliente para comunicación con el microservicio de ML.
 * 
 * @module services/ml-service-client
 */

import axios, { AxiosInstance, AxiosError } from 'axios';
import { Logger } from '../utils/logger';

// Interfaces
interface PatientData {
    age: number;
    gender: 'Male' | 'Female';
    smokingHistory: 'Never' | 'Former' | 'Current';
    yearsSmoked: number;
    packYears: number;
    bmi: number;
    lungFunctionTestResult: number;
    familyHistoryCancer: boolean;
    exposureToToxins: boolean;
    airQualityIndex: number;
    chestPainSymptoms: boolean;
    shortnessOfBreath: boolean;
    chronicCough: boolean;
    weightLoss: boolean;
    tumorSizeCm?: number;
    occupation?: string;
    residentialArea?: string;
}

interface PredictionResult {
    prediction: 'Early' | 'Advanced';
    predictionCode: number;
    probability: number;
    confidence: number;
    riskLevel: 'low' | 'medium' | 'high';
    requiresReview: boolean;
    topFactors: Array<{
        feature: string;
        value: any;
        contribution: number;
        direction: 'positive' | 'negative';
    }>;
    modelVersion: string;
    predictionId: string;
    timestamp: string;
    processingTimeMs: number;
}

interface MLServiceConfig {
    baseUrl: string;
    timeout: number;
    maxRetries: number;
    retryDelay: number;
}

/**
 * Cliente para el Microservicio de ML.
 * Implementa patrón Circuit Breaker y reintentos con backoff.
 */
class MLServiceClient {
    private client: AxiosInstance;
    private logger: Logger;
    private config: MLServiceConfig;
    private consecutiveFailures: number = 0;
    private circuitBreakerOpen: boolean = false;
    private circuitBreakerResetTime: number = 30000; // 30 segundos
    
    constructor(config: MLServiceConfig) {
        this.config = config;
        this.logger = new Logger('MLServiceClient');
        
        this.client = axios.create({
            baseURL: config.baseUrl,
            timeout: config.timeout,
            headers: {
                'Content-Type': 'application/json',
                'Accept': 'application/json'
            }
        });
        
        // Interceptor para agregar headers de tracking
        this.client.interceptors.request.use((config) => {
            config.headers['X-Request-ID'] = this.generateRequestId();
            config.headers['X-Timestamp'] = new Date().toISOString();
            return config;
        });
        
        // Interceptor para logging de respuestas
        this.client.interceptors.response.use(
            (response) => {
                this.consecutiveFailures = 0;
                return response;
            },
            (error) => {
                this.handleError(error);
                throw error;
            }
        );
    }
    
    private generateRequestId(): string {
        return `req_${Date.now()}_${Math.random().toString(36).substr(2, 9)}`;
    }
    
    private handleError(error: AxiosError): void {
        this.consecutiveFailures++;
        
        if (this.consecutiveFailures >= 5) {
            this.openCircuitBreaker();
        }
        
        this.logger.error('ML Service error', {
            status: error.response?.status,
            message: error.message,
            consecutiveFailures: this.consecutiveFailures
        });
    }
    
    private openCircuitBreaker(): void {
        this.circuitBreakerOpen = true;
        this.logger.warn('Circuit breaker OPEN - ML Service no disponible');
        
        setTimeout(() => {
            this.circuitBreakerOpen = false;
            this.consecutiveFailures = 0;
            this.logger.info('Circuit breaker CLOSED - Reintentando conexión');
        }, this.circuitBreakerResetTime);
    }
    
    /**
     * Verifica si el servicio está disponible.
     */
    async healthCheck(): Promise<boolean> {
        try {
            const response = await this.client.get('/api/v1/health');
            return response.data.status === 'healthy';
        } catch (error) {
            return false;
        }
    }
    
    /**
     * Realiza una predicción con reintentos automáticos.
     */
    async predict(patientData: PatientData): Promise<PredictionResult> {
        if (this.circuitBreakerOpen) {
            throw new Error('ML Service no disponible (Circuit Breaker activo)');
        }
        
        const transformedData = this.transformToSnakeCase(patientData);
        
        for (let attempt = 1; attempt <= this.config.maxRetries; attempt++) {
            try {
                const response = await this.client.post<PredictionResult>(
                    '/api/v1/predict',
                    transformedData
                );
                
                return this.transformToCamelCase(response.data);
                
            } catch (error) {
                if (attempt === this.config.maxRetries) {
                    throw error;
                }
                
                const delay = this.config.retryDelay * Math.pow(2, attempt - 1);
                this.logger.warn(`Reintento ${attempt}/${this.config.maxRetries} en ${delay}ms`);
                await this.sleep(delay);
            }
        }
        
        throw new Error('Máximo de reintentos alcanzado');
    }
    
    private transformToSnakeCase(data: PatientData): object {
        return {
            age: data.age,
            gender: data.gender,
            smoking_history: data.smokingHistory,
            years_smoked: data.yearsSmoked,
            pack_years: data.packYears,
            bmi: data.bmi,
            lung_function_test_result: data.lungFunctionTestResult,
            family_history_cancer: data.familyHistoryCancer,
            exposure_to_toxins: data.exposureToToxins,
            air_quality_index: data.airQualityIndex,
            chest_pain_symptoms: data.chestPainSymptoms,
            shortness_of_breath: data.shortnessOfBreath,
            chronic_cough: data.chronicCough,
            weight_loss: data.weightLoss,
            tumor_size_cm: data.tumorSizeCm,
            occupation: data.occupation,
            residential_area: data.residentialArea
        };
    }
    
    private transformToCamelCase(data: any): PredictionResult {
        return {
            prediction: data.prediction,
            predictionCode: data.prediction_code,
            probability: data.probability,
            confidence: data.confidence,
            riskLevel: data.risk_level,
            requiresReview: data.requires_review,
            topFactors: data.top_factors?.map((f: any) => ({
                feature: f.feature,
                value: f.value,
                contribution: f.contribution,
                direction: f.direction
            })) || [],
            modelVersion: data.model_version,
            predictionId: data.prediction_id,
            timestamp: data.timestamp,
            processingTimeMs: data.processing_time_ms
        };
    }
    
    private sleep(ms: number): Promise<void> {
        return new Promise(resolve => setTimeout(resolve, ms));
    }
}

// Exportar instancia configurada
export const mlServiceClient = new MLServiceClient({
    baseUrl: process.env.ML_SERVICE_URL || 'http://ml-service:8000',
    timeout: 10000,
    maxRetries: 3,
    retryDelay: 1000
});

export { MLServiceClient, PatientData, PredictionResult };
'''

print("=" * 70)
print("CLIENTE ML SERVICE PARA BACKEND NODE.JS")
print("=" * 70)
print(ml_service_client_code[:3000] + "\n... [código continúa]")

# Guardar cliente
services_path = DEPLOYMENT_DIR / 'ml_service' / 'app' / 'ml_service_client.ts'
with open(services_path, 'w', encoding='utf-8') as f:
    f.write(ml_service_client_code)
print(f"\n[OK] Cliente guardado en: {services_path}")

CLIENTE ML SERVICE PARA BACKEND NODE.JS
/**
 * LungLife Backend - ML Service Client
 * Cliente para comunicación con el microservicio de ML.
 * 
 * @module services/ml-service-client
 */

import axios, { AxiosInstance, AxiosError } from 'axios';
import { Logger } from '../utils/logger';

// Interfaces
interface PatientData {
    age: number;
    gender: 'Male' | 'Female';
    smokingHistory: 'Never' | 'Former' | 'Current';
    yearsSmoked: number;
    packYears: number;
    bmi: number;
    lungFunctionTestResult: number;
    familyHistoryCancer: boolean;
    exposureToToxins: boolean;
    airQualityIndex: number;
    chestPainSymptoms: boolean;
    shortnessOfBreath: boolean;
    chronicCough: boolean;
    weightLoss: boolean;
    tumorSizeCm?: number;
    occupation?: string;
    residentialArea?: string;
}

interface PredictionResult {
    prediction: 'Early' | 'Advanced';
    predictionCode: number;
    probability: number;
    confidence: number;
    riskLevel: 'low' | 'medium' | 

---

## 4. Persistencia de Predicciones (PostgreSQL)

### 4.1 Diagrama Entidad-Relación

El esquema de base de datos almacena predicciones y logs de auditoría para trazabilidad clínica.

In [43]:
# =============================================================================
# 4.2 DIAGRAMA ENTIDAD-RELACIÓN
# =============================================================================

er_diagram = """
╔══════════════════════════════════════════════════════════════════════════════╗
║                    DIAGRAMA ENTIDAD-RELACIÓN - LUNGLIFE                      ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  ┌────────────────────────────────────────────────────────────────────────┐  ║
║  │                              USERS                                      │  ║
║  ├────────────────────────────────────────────────────────────────────────┤  ║
║  │  PK  │ id              │ UUID         │ Identificador único            │  ║
║  │      │ email           │ VARCHAR(255) │ Email del usuario              │  ║
║  │      │ password_hash   │ VARCHAR(255) │ Hash de contraseña             │  ║
║  │      │ role            │ ENUM         │ admin, doctor, nurse           │  ║
║  │      │ full_name       │ VARCHAR(255) │ Nombre completo                │  ║
║  │      │ created_at      │ TIMESTAMP    │ Fecha de creación              │  ║
║  │      │ updated_at      │ TIMESTAMP    │ Última actualización           │  ║
║  └────────────────────────────────────────────────────────────────────────┘  ║
║                                        │                                     ║
║                                        │ 1:N                                 ║
║                                        ▼                                     ║
║  ┌────────────────────────────────────────────────────────────────────────┐  ║
║  │                            PATIENTS                                     │  ║
║  ├────────────────────────────────────────────────────────────────────────┤  ║
║  │  PK  │ id              │ UUID         │ Identificador único            │  ║
║  │  FK  │ created_by      │ UUID         │ Usuario que registró           │  ║
║  │      │ rut             │ VARCHAR(12)  │ RUT del paciente               │  ║
║  │      │ full_name       │ VARCHAR(255) │ Nombre completo                │  ║
║  │      │ age             │ INTEGER      │ Edad                           │  ║
║  │      │ gender          │ ENUM         │ Male, Female                   │  ║
║  │      │ region          │ VARCHAR(100) │ Región de residencia           │  ║
║  │      │ created_at      │ TIMESTAMP    │ Fecha de registro              │  ║
║  └────────────────────────────────────────────────────────────────────────┘  ║
║                                        │                                     ║
║                                        │ 1:N                                 ║
║                                        ▼                                     ║
║  ┌────────────────────────────────────────────────────────────────────────┐  ║
║  │                          PREDICTIONS                                    │  ║
║  ├────────────────────────────────────────────────────────────────────────┤  ║
║  │  PK  │ id              │ UUID         │ Identificador único            │  ║
║  │  FK  │ patient_id      │ UUID         │ Paciente asociado              │  ║
║  │  FK  │ requested_by    │ UUID         │ Usuario que solicitó           │  ║
║  │      │ prediction_id   │ VARCHAR(50)  │ ID externo de predicción       │  ║
║  │      │ prediction      │ ENUM         │ Early, Advanced                │  ║
║  │      │ probability     │ DECIMAL(5,4) │ Probabilidad (0-1)             │  ║
║  │      │ confidence      │ DECIMAL(5,4) │ Confianza (0-1)                │  ║
║  │      │ risk_level      │ ENUM         │ low, medium, high              │  ║
║  │      │ requires_review │ BOOLEAN      │ Requiere revisión médica       │  ║
║  │      │ model_version   │ VARCHAR(20)  │ Versión del modelo             │  ║
║  │      │ input_data      │ JSONB        │ Datos de entrada (encriptados) │  ║
║  │      │ top_factors     │ JSONB        │ Factores contribuyentes        │  ║
║  │      │ processing_ms   │ DECIMAL(10,2)│ Tiempo de procesamiento        │  ║
║  │      │ created_at      │ TIMESTAMP    │ Fecha de predicción            │  ║
║  └────────────────────────────────────────────────────────────────────────┘  ║
║                                        │                                     ║
║                                        │ 1:N                                 ║
║                                        ▼                                     ║
║  ┌────────────────────────────────────────────────────────────────────────┐  ║
║  │                          AUDIT_LOGS                                     │  ║
║  ├────────────────────────────────────────────────────────────────────────┤  ║
║  │  PK  │ id              │ UUID         │ Identificador único            │  ║
║  │  FK  │ user_id         │ UUID         │ Usuario que realizó acción     │  ║
║  │  FK  │ prediction_id   │ UUID         │ Predicción asociada (opcional) │  ║
║  │      │ action          │ ENUM         │ CREATE, READ, UPDATE, DELETE   │  ║
║  │      │ entity_type     │ VARCHAR(50)  │ Tipo de entidad afectada       │  ║
║  │      │ entity_id       │ UUID         │ ID de la entidad               │  ║
║  │      │ old_values      │ JSONB        │ Valores anteriores             │  ║
║  │      │ new_values      │ JSONB        │ Valores nuevos                 │  ║
║  │      │ ip_address      │ INET         │ Dirección IP del cliente       │  ║
║  │      │ user_agent      │ TEXT         │ User agent del navegador       │  ║
║  │      │ created_at      │ TIMESTAMP    │ Fecha del evento               │  ║
║  └────────────────────────────────────────────────────────────────────────┘  ║
║                                                                              ║
╚══════════════════════════════════════════════════════════════════════════════╝
"""

print(er_diagram)

# Guardar diagrama
er_path = DEPLOYMENT_DIR / 'docs' / 'er_diagram.txt'
with open(er_path, 'w', encoding='utf-8') as f:
    f.write(er_diagram)
print(f"\n[OK] Diagrama ER guardado en: {er_path}")


╔══════════════════════════════════════════════════════════════════════════════╗
║                    DIAGRAMA ENTIDAD-RELACIÓN - LUNGLIFE                      ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  ┌────────────────────────────────────────────────────────────────────────┐  ║
║  │                              USERS                                      │  ║
║  ├────────────────────────────────────────────────────────────────────────┤  ║
║  │  PK  │ id              │ UUID         │ Identificador único            │  ║
║  │      │ email           │ VARCHAR(255) │ Email del usuario              │  ║
║  │      │ password_hash   │ VARCHAR(255) │ Hash de contraseña             │  ║
║  │      │ role            │ ENUM         │ admin, doctor, nurse           │  ║
║  │      │ full_name       │ VARCHAR(255) │ Nombre completo                │  ║
║  │      │ created_at    

In [44]:
# =============================================================================
# 4.3 ESQUEMA SQL DE POSTGRESQL
# =============================================================================

postgres_schema = '''-- =============================================================================
-- LungLife Database Schema
-- PostgreSQL 15+
-- =============================================================================

-- Extensiones necesarias
CREATE EXTENSION IF NOT EXISTS "uuid-ossp";
CREATE EXTENSION IF NOT EXISTS "pgcrypto";

-- =============================================================================
-- ENUMS
-- =============================================================================

CREATE TYPE user_role AS ENUM ('admin', 'doctor', 'nurse', 'technician');
CREATE TYPE gender_type AS ENUM ('Male', 'Female');
CREATE TYPE prediction_result AS ENUM ('Early', 'Advanced');
CREATE TYPE risk_level_type AS ENUM ('low', 'medium', 'high');
CREATE TYPE audit_action AS ENUM ('CREATE', 'READ', 'UPDATE', 'DELETE', 'PREDICT');

-- =============================================================================
-- TABLA: users
-- Usuarios del sistema (médicos, enfermeras, técnicos)
-- =============================================================================

CREATE TABLE users (
    id UUID PRIMARY KEY DEFAULT uuid_generate_v4(),
    email VARCHAR(255) UNIQUE NOT NULL,
    password_hash VARCHAR(255) NOT NULL,
    role user_role NOT NULL DEFAULT 'doctor',
    full_name VARCHAR(255) NOT NULL,
    professional_license VARCHAR(50),
    department VARCHAR(100),
    is_active BOOLEAN DEFAULT TRUE,
    last_login TIMESTAMP WITH TIME ZONE,
    created_at TIMESTAMP WITH TIME ZONE DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP WITH TIME ZONE DEFAULT CURRENT_TIMESTAMP
);

CREATE INDEX idx_users_email ON users(email);
CREATE INDEX idx_users_role ON users(role);

-- =============================================================================
-- TABLA: patients
-- Registro de pacientes
-- =============================================================================

CREATE TABLE patients (
    id UUID PRIMARY KEY DEFAULT uuid_generate_v4(),
    created_by UUID NOT NULL REFERENCES users(id),
    rut VARCHAR(12) UNIQUE NOT NULL,
    full_name VARCHAR(255) NOT NULL,
    birth_date DATE,
    age INTEGER CHECK (age >= 0 AND age <= 150),
    gender gender_type NOT NULL,
    phone VARCHAR(20),
    email VARCHAR(255),
    address TEXT,
    region VARCHAR(100),
    commune VARCHAR(100),
    emergency_contact_name VARCHAR(255),
    emergency_contact_phone VARCHAR(20),
    medical_record_number VARCHAR(50),
    created_at TIMESTAMP WITH TIME ZONE DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP WITH TIME ZONE DEFAULT CURRENT_TIMESTAMP
);

CREATE INDEX idx_patients_rut ON patients(rut);
CREATE INDEX idx_patients_region ON patients(region);
CREATE INDEX idx_patients_created_by ON patients(created_by);

-- =============================================================================
-- TABLA: predictions
-- Registro de predicciones del modelo ML
-- =============================================================================

CREATE TABLE predictions (
    id UUID PRIMARY KEY DEFAULT uuid_generate_v4(),
    patient_id UUID NOT NULL REFERENCES patients(id) ON DELETE CASCADE,
    requested_by UUID NOT NULL REFERENCES users(id),
    
    -- Identificador externo de la predicción
    prediction_id VARCHAR(50) UNIQUE NOT NULL,
    
    -- Resultados del modelo
    prediction prediction_result NOT NULL,
    probability DECIMAL(5, 4) NOT NULL CHECK (probability >= 0 AND probability <= 1),
    confidence DECIMAL(5, 4) NOT NULL CHECK (confidence >= 0 AND confidence <= 1),
    risk_level risk_level_type NOT NULL,
    requires_review BOOLEAN DEFAULT FALSE,
    
    -- Metadatos del modelo
    model_version VARCHAR(20) NOT NULL,
    threshold_used DECIMAL(3, 2) DEFAULT 0.50,
    
    -- Datos de entrada (encriptados para HIPAA/GDPR)
    input_data JSONB NOT NULL,
    input_data_hash VARCHAR(64), -- SHA-256 para verificación de integridad
    
    -- Interpretabilidad (SHAP)
    top_factors JSONB,
    
    -- Métricas de rendimiento
    processing_time_ms DECIMAL(10, 2),
    
    -- Estado de revisión
    reviewed_by UUID REFERENCES users(id),
    reviewed_at TIMESTAMP WITH TIME ZONE,
    review_notes TEXT,
    
    -- Timestamps
    created_at TIMESTAMP WITH TIME ZONE DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP WITH TIME ZONE DEFAULT CURRENT_TIMESTAMP
);

CREATE INDEX idx_predictions_patient ON predictions(patient_id);
CREATE INDEX idx_predictions_requested_by ON predictions(requested_by);
CREATE INDEX idx_predictions_created_at ON predictions(created_at DESC);
CREATE INDEX idx_predictions_risk_level ON predictions(risk_level);
CREATE INDEX idx_predictions_prediction ON predictions(prediction);

-- =============================================================================
-- TABLA: audit_logs
-- Registro de auditoría para trazabilidad clínica
-- =============================================================================

CREATE TABLE audit_logs (
    id UUID PRIMARY KEY DEFAULT uuid_generate_v4(),
    user_id UUID REFERENCES users(id),
    prediction_id UUID REFERENCES predictions(id) ON DELETE SET NULL,
    
    -- Información de la acción
    action audit_action NOT NULL,
    entity_type VARCHAR(50) NOT NULL,
    entity_id UUID,
    
    -- Datos de cambios
    old_values JSONB,
    new_values JSONB,
    
    -- Información del cliente
    ip_address INET,
    user_agent TEXT,
    request_id VARCHAR(100),
    
    -- Información adicional
    description TEXT,
    metadata JSONB,
    
    -- Timestamp
    created_at TIMESTAMP WITH TIME ZONE DEFAULT CURRENT_TIMESTAMP
);

CREATE INDEX idx_audit_logs_user ON audit_logs(user_id);
CREATE INDEX idx_audit_logs_action ON audit_logs(action);
CREATE INDEX idx_audit_logs_entity ON audit_logs(entity_type, entity_id);
CREATE INDEX idx_audit_logs_created_at ON audit_logs(created_at DESC);

-- =============================================================================
-- FUNCIONES Y TRIGGERS
-- =============================================================================

-- Función para actualizar updated_at automáticamente
CREATE OR REPLACE FUNCTION update_updated_at_column()
RETURNS TRIGGER AS $$
BEGIN
    NEW.updated_at = CURRENT_TIMESTAMP;
    RETURN NEW;
END;
$$ language 'plpgsql';

-- Triggers para updated_at
CREATE TRIGGER update_users_updated_at
    BEFORE UPDATE ON users
    FOR EACH ROW
    EXECUTE FUNCTION update_updated_at_column();

CREATE TRIGGER update_patients_updated_at
    BEFORE UPDATE ON patients
    FOR EACH ROW
    EXECUTE FUNCTION update_updated_at_column();

CREATE TRIGGER update_predictions_updated_at
    BEFORE UPDATE ON predictions
    FOR EACH ROW
    EXECUTE FUNCTION update_updated_at_column();

-- Función para crear log de auditoría automático
CREATE OR REPLACE FUNCTION create_audit_log()
RETURNS TRIGGER AS $$
DECLARE
    action_type audit_action;
    old_data JSONB;
    new_data JSONB;
BEGIN
    IF TG_OP = 'INSERT' THEN
        action_type := 'CREATE';
        old_data := NULL;
        new_data := to_jsonb(NEW);
    ELSIF TG_OP = 'UPDATE' THEN
        action_type := 'UPDATE';
        old_data := to_jsonb(OLD);
        new_data := to_jsonb(NEW);
    ELSIF TG_OP = 'DELETE' THEN
        action_type := 'DELETE';
        old_data := to_jsonb(OLD);
        new_data := NULL;
    END IF;
    
    INSERT INTO audit_logs (
        user_id,
        action,
        entity_type,
        entity_id,
        old_values,
        new_values
    ) VALUES (
        COALESCE(NEW.requested_by, NEW.created_by, OLD.requested_by, OLD.created_by),
        action_type,
        TG_TABLE_NAME,
        COALESCE(NEW.id, OLD.id),
        old_data,
        new_data
    );
    
    RETURN COALESCE(NEW, OLD);
END;
$$ LANGUAGE plpgsql;

-- Trigger para auditoría de predicciones
CREATE TRIGGER audit_predictions
    AFTER INSERT OR UPDATE OR DELETE ON predictions
    FOR EACH ROW
    EXECUTE FUNCTION create_audit_log();

-- =============================================================================
-- VISTAS
-- =============================================================================

-- Vista de predicciones con información del paciente
CREATE VIEW vw_predictions_summary AS
SELECT 
    p.id,
    p.prediction_id,
    p.prediction,
    p.probability,
    p.risk_level,
    p.created_at,
    pt.rut AS patient_rut,
    pt.full_name AS patient_name,
    pt.region AS patient_region,
    u.full_name AS requested_by_name
FROM predictions p
JOIN patients pt ON p.patient_id = pt.id
JOIN users u ON p.requested_by = u.id;

-- Vista de estadísticas diarias
CREATE VIEW vw_daily_statistics AS
SELECT 
    DATE(created_at) AS date,
    COUNT(*) AS total_predictions,
    SUM(CASE WHEN prediction = 'Advanced' THEN 1 ELSE 0 END) AS advanced_count,
    SUM(CASE WHEN prediction = 'Early' THEN 1 ELSE 0 END) AS early_count,
    AVG(probability) AS avg_probability,
    AVG(processing_time_ms) AS avg_processing_time
FROM predictions
GROUP BY DATE(created_at)
ORDER BY date DESC;
'''

print("=" * 70)
print("ESQUEMA SQL DE POSTGRESQL")
print("=" * 70)
print(postgres_schema[:3000] + "\n... [esquema continúa]")

# Guardar esquema
schema_path = DEPLOYMENT_DIR / 'schemas' / 'database_schema.sql'
with open(schema_path, 'w', encoding='utf-8') as f:
    f.write(postgres_schema)
print(f"\n[OK] Esquema SQL guardado en: {schema_path}")

ESQUEMA SQL DE POSTGRESQL
-- =============================================================================
-- LungLife Database Schema
-- PostgreSQL 15+
-- =============================================================================

-- Extensiones necesarias
CREATE EXTENSION IF NOT EXISTS "uuid-ossp";
CREATE EXTENSION IF NOT EXISTS "pgcrypto";

-- =============================================================================
-- ENUMS
-- =============================================================================

CREATE TYPE user_role AS ENUM ('admin', 'doctor', 'nurse', 'technician');
CREATE TYPE gender_type AS ENUM ('Male', 'Female');
CREATE TYPE prediction_result AS ENUM ('Early', 'Advanced');
CREATE TYPE risk_level_type AS ENUM ('low', 'medium', 'high');
CREATE TYPE audit_action AS ENUM ('CREATE', 'READ', 'UPDATE', 'DELETE', 'PREDICT');

-- =============================================================================
-- TABLA: users
-- Usuarios del sistema (médicos, enfermeras, t

---

## 5. Contenerización (Docker)

### 5.1 Diagrama de Contenedores Docker

La arquitectura de contenedores permite desplegar todos los servicios con un solo comando.

In [45]:
# =============================================================================
# 5.2 DIAGRAMA DE CONTENEDORES DOCKER
# =============================================================================

docker_diagram = """
╔══════════════════════════════════════════════════════════════════════════════╗
║                    ARQUITECTURA DE CONTENEDORES DOCKER                       ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  ┌────────────────────────────────────────────────────────────────────────┐  ║
║  │                         DOCKER NETWORK: lunglife-network               │  ║
║  │                              (bridge mode)                              │  ║
║  ├────────────────────────────────────────────────────────────────────────┤  ║
║  │                                                                        │  ║
║  │  ┌─────────────────┐  ┌─────────────────┐  ┌─────────────────┐        │  ║
║  │  │   frontend      │  │   backend       │  │   ml-service    │        │  ║
║  │  │   (Ionic/Nginx) │  │   (Node.js)     │  │   (FastAPI)     │        │  ║
║  │  │                 │  │                 │  │                 │        │  ║
║  │  │  Port: 8100     │  │  Port: 3000     │  │  Port: 8000     │        │  ║
║  │  │  Image: nginx   │  │  Image: node:20 │  │  Image: python  │        │  ║
║  │  │  Memory: 256MB  │  │  Memory: 512MB  │  │  Memory: 1GB    │        │  ║
║  │  │                 │  │                 │  │                 │        │  ║
║  │  │  Volumes:       │  │  Volumes:       │  │  Volumes:       │        │  ║
║  │  │  - dist/        │  │  - node_modules │  │  - models/      │        │  ║
║  │  │  - nginx.conf   │  │  - logs/        │  │  - logs/        │        │  ║
║  │  └────────┬────────┘  └────────┬────────┘  └────────┬────────┘        │  ║
║  │           │                    │                    │                  │  ║
║  │           └────────────────────┼────────────────────┘                  │  ║
║  │                                │                                       │  ║
║  │  ┌─────────────────────────────┼─────────────────────────────┐        │  ║
║  │  │                             ▼                             │        │  ║
║  │  │  ┌─────────────────┐  ┌─────────────────┐                │        │  ║
║  │  │  │   postgres      │  │   redis         │                │        │  ║
║  │  │  │   (Database)    │  │   (Cache)       │                │        │  ║
║  │  │  │                 │  │                 │                │        │  ║
║  │  │  │  Port: 5432     │  │  Port: 6379     │                │        │  ║
║  │  │  │  Image: pg:15   │  │  Image: redis:7 │                │        │  ║
║  │  │  │  Memory: 512MB  │  │  Memory: 256MB  │                │        │  ║
║  │  │  │                 │  │                 │                │        │  ║
║  │  │  │  Volumes:       │  │  Volumes:       │                │        │  ║
║  │  │  │  - pgdata/      │  │  - redisdata/   │                │        │  ║
║  │  │  └─────────────────┘  └─────────────────┘                │        │  ║
║  │  │                   DATA LAYER                              │        │  ║
║  │  └───────────────────────────────────────────────────────────┘        │  ║
║  │                                                                        │  ║
║  └────────────────────────────────────────────────────────────────────────┘  ║
║                                                                              ║
║  ┌────────────────────────────────────────────────────────────────────────┐  ║
║  │                          DOCKER VOLUMES                                │  ║
║  ├────────────────────────────────────────────────────────────────────────┤  ║
║  │  pgdata          │ PostgreSQL data persistence                         │  ║
║  │  redisdata       │ Redis data persistence                              │  ║
║  │  ml-models       │ ML model artifacts                                  │  ║
║  │  logs            │ Application logs                                    │  ║
║  └────────────────────────────────────────────────────────────────────────┘  ║
║                                                                              ║
╚══════════════════════════════════════════════════════════════════════════════╝
"""

print(docker_diagram)

# Guardar diagrama
docker_diag_path = DEPLOYMENT_DIR / 'docs' / 'docker_architecture.txt'
with open(docker_diag_path, 'w', encoding='utf-8') as f:
    f.write(docker_diagram)
print(f"\n[OK] Diagrama Docker guardado en: {docker_diag_path}")


╔══════════════════════════════════════════════════════════════════════════════╗
║                    ARQUITECTURA DE CONTENEDORES DOCKER                       ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  ┌────────────────────────────────────────────────────────────────────────┐  ║
║  │                         DOCKER NETWORK: lunglife-network               │  ║
║  │                              (bridge mode)                              │  ║
║  ├────────────────────────────────────────────────────────────────────────┤  ║
║  │                                                                        │  ║
║  │  ┌─────────────────┐  ┌─────────────────┐  ┌─────────────────┐        │  ║
║  │  │   frontend      │  │   backend       │  │   ml-service    │        │  ║
║  │  │   (Ionic/Nginx) │  │   (Node.js)     │  │   (FastAPI)     │        │  ║
║  │  │                 │  │ 

In [46]:
# =============================================================================
# 5.3 DOCKER-COMPOSE.YML - ORQUESTACIÓN COMPLETA
# =============================================================================

docker_compose_content = '''# ============================================================================
# LungLife MVP - Docker Compose Configuration
# Versión: 1.0.0
# Autor: LungLife Team
# Descripción: Orquestación de servicios para el MVP de predicción de cáncer
# ============================================================================

version: '3.8'

# =============================================================================
# NETWORKS
# =============================================================================
networks:
  lunglife-network:
    driver: bridge
    ipam:
      config:
        - subnet: 172.28.0.0/16

# =============================================================================
# VOLUMES
# =============================================================================
volumes:
  pgdata:
    name: lunglife_pgdata
  redisdata:
    name: lunglife_redisdata
  ml-models:
    name: lunglife_ml_models
  logs:
    name: lunglife_logs

# =============================================================================
# SERVICES
# =============================================================================
services:
  # ---------------------------------------------------------------------------
  # PostgreSQL Database
  # ---------------------------------------------------------------------------
  postgres:
    image: postgres:15-alpine
    container_name: lunglife_postgres
    restart: unless-stopped
    environment:
      POSTGRES_DB: ${POSTGRES_DB:-lunglife}
      POSTGRES_USER: ${POSTGRES_USER:-lunglife_user}
      POSTGRES_PASSWORD: ${POSTGRES_PASSWORD:-secure_password_change_me}
      PGDATA: /var/lib/postgresql/data/pgdata
    volumes:
      - pgdata:/var/lib/postgresql/data
      - ./database/init:/docker-entrypoint-initdb.d:ro
    ports:
      - "${POSTGRES_PORT:-5432}:5432"
    networks:
      lunglife-network:
        ipv4_address: 172.28.0.10
    healthcheck:
      test: ["CMD-SHELL", "pg_isready -U ${POSTGRES_USER:-lunglife_user} -d ${POSTGRES_DB:-lunglife}"]
      interval: 10s
      timeout: 5s
      retries: 5
      start_period: 30s
    deploy:
      resources:
        limits:
          memory: 512M
        reservations:
          memory: 256M

  # ---------------------------------------------------------------------------
  # Redis Cache
  # ---------------------------------------------------------------------------
  redis:
    image: redis:7-alpine
    container_name: lunglife_redis
    restart: unless-stopped
    command: redis-server --appendonly yes --maxmemory 256mb --maxmemory-policy allkeys-lru
    volumes:
      - redisdata:/data
    ports:
      - "${REDIS_PORT:-6379}:6379"
    networks:
      lunglife-network:
        ipv4_address: 172.28.0.11
    healthcheck:
      test: ["CMD", "redis-cli", "ping"]
      interval: 10s
      timeout: 5s
      retries: 5
    deploy:
      resources:
        limits:
          memory: 256M
        reservations:
          memory: 128M

  # ---------------------------------------------------------------------------
  # ML Service (FastAPI)
  # ---------------------------------------------------------------------------
  ml-service:
    build:
      context: ./ml_service
      dockerfile: Dockerfile
    container_name: lunglife_ml_service
    restart: unless-stopped
    environment:
      - ENVIRONMENT=${ENVIRONMENT:-production}
      - MODEL_PATH=/app/models
      - LOG_LEVEL=${LOG_LEVEL:-INFO}
      - REDIS_URL=redis://redis:6379/0
      - MAX_WORKERS=${ML_MAX_WORKERS:-4}
      - MODEL_VERSION=${MODEL_VERSION:-1.0.0}
    volumes:
      - ml-models:/app/models:ro
      - logs:/app/logs
    ports:
      - "${ML_SERVICE_PORT:-8000}:8000"
    networks:
      lunglife-network:
        ipv4_address: 172.28.0.20
    depends_on:
      redis:
        condition: service_healthy
    healthcheck:
      test: ["CMD", "curl", "-f", "http://localhost:8000/health"]
      interval: 30s
      timeout: 10s
      retries: 3
      start_period: 60s
    deploy:
      resources:
        limits:
          memory: 1G
        reservations:
          memory: 512M

  # ---------------------------------------------------------------------------
  # Backend (Node.js/Express)
  # ---------------------------------------------------------------------------
  backend:
    build:
      context: ./backend
      dockerfile: Dockerfile
    container_name: lunglife_backend
    restart: unless-stopped
    environment:
      - NODE_ENV=${NODE_ENV:-production}
      - PORT=3000
      - DATABASE_URL=postgresql://${POSTGRES_USER:-lunglife_user}:${POSTGRES_PASSWORD:-secure_password_change_me}@postgres:5432/${POSTGRES_DB:-lunglife}
      - REDIS_URL=redis://redis:6379/0
      - ML_SERVICE_URL=http://ml-service:8000
      - JWT_SECRET=${JWT_SECRET:-change_this_secret_in_production}
      - JWT_EXPIRES_IN=${JWT_EXPIRES_IN:-24h}
      - LOG_LEVEL=${LOG_LEVEL:-info}
    volumes:
      - logs:/app/logs
    ports:
      - "${BACKEND_PORT:-3000}:3000"
    networks:
      lunglife-network:
        ipv4_address: 172.28.0.30
    depends_on:
      postgres:
        condition: service_healthy
      redis:
        condition: service_healthy
      ml-service:
        condition: service_healthy
    healthcheck:
      test: ["CMD", "curl", "-f", "http://localhost:3000/api/health"]
      interval: 30s
      timeout: 10s
      retries: 3
      start_period: 30s
    deploy:
      resources:
        limits:
          memory: 512M
        reservations:
          memory: 256M

  # ---------------------------------------------------------------------------
  # Frontend (Ionic/Angular con Nginx)
  # ---------------------------------------------------------------------------
  frontend:
    build:
      context: ./frontend
      dockerfile: Dockerfile
      args:
        - API_URL=${API_URL:-http://localhost:3000/api}
    container_name: lunglife_frontend
    restart: unless-stopped
    ports:
      - "${FRONTEND_PORT:-8100}:80"
    networks:
      lunglife-network:
        ipv4_address: 172.28.0.40
    depends_on:
      - backend
    healthcheck:
      test: ["CMD", "curl", "-f", "http://localhost:80"]
      interval: 30s
      timeout: 10s
      retries: 3
    deploy:
      resources:
        limits:
          memory: 256M
        reservations:
          memory: 128M
'''

print("=" * 80)
print("DOCKER-COMPOSE.YML - CONFIGURACIÓN COMPLETA")
print("=" * 80)
print(docker_compose_content)

# Guardar archivo
compose_path = DEPLOYMENT_DIR / 'docker-compose.yml'
with open(compose_path, 'w', encoding='utf-8') as f:
    f.write(docker_compose_content)
print(f"\n[OK] docker-compose.yml guardado en: {compose_path}")

DOCKER-COMPOSE.YML - CONFIGURACIÓN COMPLETA
# ============================================================================
# LungLife MVP - Docker Compose Configuration
# Versión: 1.0.0
# Autor: LungLife Team
# Descripción: Orquestación de servicios para el MVP de predicción de cáncer
# ============================================================================

version: '3.8'

# =============================================================================
# NETWORKS
# =============================================================================
networks:
  lunglife-network:
    driver: bridge
    ipam:
      config:
        - subnet: 172.28.0.0/16

# =============================================================================
# VOLUMES
# =============================================================================
volumes:
  pgdata:
    name: lunglife_pgdata
  redisdata:
    name: lunglife_redisdata
  ml-models:
    name: lunglife_ml_models
  logs:
    name: lunglife_logs

# =======

In [47]:
# =============================================================================
# 5.4 DOCKERFILE - ML SERVICE (FastAPI)
# =============================================================================

ml_service_dockerfile = '''# ============================================================================
# LungLife ML Service - Dockerfile
# Base: Python 3.11 slim
# Purpose: FastAPI ML inference service
# ============================================================================

# -----------------------------------------------------------------------------
# Stage 1: Builder
# -----------------------------------------------------------------------------
FROM python:3.11-slim as builder

WORKDIR /build

# Install build dependencies
RUN apt-get update && apt-get install -y --no-install-recommends \\
    build-essential \\
    curl \\
    && rm -rf /var/lib/apt/lists/*

# Create virtual environment
RUN python -m venv /opt/venv
ENV PATH="/opt/venv/bin:$PATH"

# Copy and install requirements
COPY requirements.txt .
RUN pip install --no-cache-dir --upgrade pip && \\
    pip install --no-cache-dir -r requirements.txt

# -----------------------------------------------------------------------------
# Stage 2: Production
# -----------------------------------------------------------------------------
FROM python:3.11-slim as production

# Labels
LABEL maintainer="LungLife Team"
LABEL version="1.0.0"
LABEL description="LungLife ML Inference Service"

# Create non-root user
RUN groupadd --gid 1000 lunglife && \\
    useradd --uid 1000 --gid lunglife --shell /bin/bash --create-home lunglife

WORKDIR /app

# Install runtime dependencies
RUN apt-get update && apt-get install -y --no-install-recommends \\
    curl \\
    && rm -rf /var/lib/apt/lists/* \\
    && apt-get clean

# Copy virtual environment from builder
COPY --from=builder /opt/venv /opt/venv
ENV PATH="/opt/venv/bin:$PATH"

# Copy application code
COPY --chown=lunglife:lunglife ./app /app/app
COPY --chown=lunglife:lunglife ./models /app/models

# Create logs directory
RUN mkdir -p /app/logs && chown -R lunglife:lunglife /app/logs

# Switch to non-root user
USER lunglife

# Environment variables
ENV PYTHONDONTWRITEBYTECODE=1
ENV PYTHONUNBUFFERED=1
ENV ENVIRONMENT=production
ENV MODEL_PATH=/app/models
ENV LOG_LEVEL=INFO

# Expose port
EXPOSE 8000

# Health check
HEALTHCHECK --interval=30s --timeout=10s --start-period=60s --retries=3 \\
    CMD curl -f http://localhost:8000/health || exit 1

# Start application
CMD ["uvicorn", "app.main:app", "--host", "0.0.0.0", "--port", "8000", "--workers", "4"]
'''

print("=" * 80)
print("DOCKERFILE - ML SERVICE (FastAPI)")
print("=" * 80)
print(ml_service_dockerfile)

# Guardar archivo
ml_dockerfile_path = DEPLOYMENT_DIR / 'ml_service' / 'Dockerfile'
with open(ml_dockerfile_path, 'w', encoding='utf-8') as f:
    f.write(ml_service_dockerfile)
print(f"\n[OK] Dockerfile ML Service guardado en: {ml_dockerfile_path}")

DOCKERFILE - ML SERVICE (FastAPI)
# ============================================================================
# LungLife ML Service - Dockerfile
# Base: Python 3.11 slim
# Purpose: FastAPI ML inference service
# ============================================================================

# -----------------------------------------------------------------------------
# Stage 1: Builder
# -----------------------------------------------------------------------------
FROM python:3.11-slim as builder

WORKDIR /build

# Install build dependencies
RUN apt-get update && apt-get install -y --no-install-recommends \
    build-essential \
    curl \
    && rm -rf /var/lib/apt/lists/*

# Create virtual environment
RUN python -m venv /opt/venv
ENV PATH="/opt/venv/bin:$PATH"

# Copy and install requirements
COPY requirements.txt .
RUN pip install --no-cache-dir --upgrade pip && \
    pip install --no-cache-dir -r requirements.txt

# ---------------------------------------------------------------

In [48]:
# =============================================================================
# 5.5 DOCKERFILE - BACKEND (Node.js/Express)
# =============================================================================

backend_dockerfile = '''# ============================================================================
# LungLife Backend - Dockerfile
# Base: Node.js 20 Alpine
# Purpose: Express API Gateway
# ============================================================================

# -----------------------------------------------------------------------------
# Stage 1: Builder
# -----------------------------------------------------------------------------
FROM node:20-alpine as builder

WORKDIR /build

# Install dependencies for native modules
RUN apk add --no-cache python3 make g++

# Copy package files
COPY package*.json ./

# Install all dependencies (including devDependencies for build)
RUN npm ci

# Copy source code
COPY . .

# Build TypeScript
RUN npm run build

# Prune devDependencies
RUN npm prune --production

# -----------------------------------------------------------------------------
# Stage 2: Production
# -----------------------------------------------------------------------------
FROM node:20-alpine as production

# Labels
LABEL maintainer="LungLife Team"
LABEL version="1.0.0"
LABEL description="LungLife Backend API Gateway"

# Create non-root user
RUN addgroup -g 1000 lunglife && \\
    adduser -u 1000 -G lunglife -s /bin/sh -D lunglife

WORKDIR /app

# Install runtime dependencies
RUN apk add --no-cache curl dumb-init

# Copy built application from builder
COPY --from=builder --chown=lunglife:lunglife /build/dist ./dist
COPY --from=builder --chown=lunglife:lunglife /build/node_modules ./node_modules
COPY --from=builder --chown=lunglife:lunglife /build/package*.json ./

# Create logs directory
RUN mkdir -p /app/logs && chown -R lunglife:lunglife /app/logs

# Switch to non-root user
USER lunglife

# Environment variables
ENV NODE_ENV=production
ENV PORT=3000

# Expose port
EXPOSE 3000

# Health check
HEALTHCHECK --interval=30s --timeout=10s --start-period=30s --retries=3 \\
    CMD curl -f http://localhost:3000/api/health || exit 1

# Use dumb-init for proper signal handling
ENTRYPOINT ["dumb-init", "--"]

# Start application
CMD ["node", "dist/server.js"]
'''

print("=" * 80)
print("DOCKERFILE - BACKEND (Node.js/Express)")
print("=" * 80)
print(backend_dockerfile)

# Guardar archivo
backend_dockerfile_path = DEPLOYMENT_DIR / 'backend' / 'Dockerfile'
backend_dockerfile_path.parent.mkdir(parents=True, exist_ok=True)
with open(backend_dockerfile_path, 'w', encoding='utf-8') as f:
    f.write(backend_dockerfile)
print(f"\n[OK] Dockerfile Backend guardado en: {backend_dockerfile_path}")

DOCKERFILE - BACKEND (Node.js/Express)
# ============================================================================
# LungLife Backend - Dockerfile
# Base: Node.js 20 Alpine
# Purpose: Express API Gateway
# ============================================================================

# -----------------------------------------------------------------------------
# Stage 1: Builder
# -----------------------------------------------------------------------------
FROM node:20-alpine as builder

WORKDIR /build

# Install dependencies for native modules
RUN apk add --no-cache python3 make g++

# Copy package files
COPY package*.json ./

# Install all dependencies (including devDependencies for build)
RUN npm ci

# Copy source code
COPY . .

# Build TypeScript
RUN npm run build

# Prune devDependencies
RUN npm prune --production

# -----------------------------------------------------------------------------
# Stage 2: Production
# -----------------------------------------------------------

In [49]:
# =============================================================================
# 5.6 DOCKERFILE - FRONTEND (Ionic/Angular + Nginx)
# =============================================================================

frontend_dockerfile = '''# ============================================================================
# LungLife Frontend - Dockerfile
# Base: Node.js 20 (build) + Nginx Alpine (serve)
# Purpose: Ionic/Angular SPA with Nginx
# ============================================================================

# -----------------------------------------------------------------------------
# Stage 1: Builder
# -----------------------------------------------------------------------------
FROM node:20-alpine as builder

ARG API_URL=http://localhost:3000/api

WORKDIR /build

# Install dependencies for native modules
RUN apk add --no-cache python3 make g++

# Copy package files
COPY package*.json ./

# Install dependencies
RUN npm ci

# Copy source code
COPY . .

# Set environment for build
ENV IONIC_CLI_VERSION=7.1.1
RUN npm install -g @ionic/cli@${IONIC_CLI_VERSION}

# Build for production
RUN npm run build -- --configuration=production

# -----------------------------------------------------------------------------
# Stage 2: Production (Nginx)
# -----------------------------------------------------------------------------
FROM nginx:alpine as production

# Labels
LABEL maintainer="LungLife Team"
LABEL version="1.0.0"
LABEL description="LungLife Frontend PWA"

# Remove default nginx config
RUN rm -rf /etc/nginx/conf.d/*

# Copy custom nginx configuration
COPY nginx.conf /etc/nginx/conf.d/default.conf

# Copy built application
COPY --from=builder /build/www /usr/share/nginx/html

# Create non-root user for nginx
RUN chown -R nginx:nginx /usr/share/nginx/html && \\
    chown -R nginx:nginx /var/cache/nginx && \\
    chown -R nginx:nginx /var/log/nginx && \\
    touch /var/run/nginx.pid && \\
    chown -R nginx:nginx /var/run/nginx.pid

# Expose port
EXPOSE 80

# Health check
HEALTHCHECK --interval=30s --timeout=10s --retries=3 \\
    CMD curl -f http://localhost:80 || exit 1

# Start Nginx
CMD ["nginx", "-g", "daemon off;"]
'''

print("=" * 80)
print("DOCKERFILE - FRONTEND (Ionic/Angular + Nginx)")
print("=" * 80)
print(frontend_dockerfile)

# Guardar archivo
frontend_dockerfile_path = DEPLOYMENT_DIR / 'frontend' / 'Dockerfile'
frontend_dockerfile_path.parent.mkdir(parents=True, exist_ok=True)
with open(frontend_dockerfile_path, 'w', encoding='utf-8') as f:
    f.write(frontend_dockerfile)
print(f"\n[OK] Dockerfile Frontend guardado en: {frontend_dockerfile_path}")

DOCKERFILE - FRONTEND (Ionic/Angular + Nginx)
# ============================================================================
# LungLife Frontend - Dockerfile
# Base: Node.js 20 (build) + Nginx Alpine (serve)
# Purpose: Ionic/Angular SPA with Nginx
# ============================================================================

# -----------------------------------------------------------------------------
# Stage 1: Builder
# -----------------------------------------------------------------------------
FROM node:20-alpine as builder

ARG API_URL=http://localhost:3000/api

WORKDIR /build

# Install dependencies for native modules
RUN apk add --no-cache python3 make g++

# Copy package files
COPY package*.json ./

# Install dependencies
RUN npm ci

# Copy source code
COPY . .

# Set environment for build
ENV IONIC_CLI_VERSION=7.1.1
RUN npm install -g @ionic/cli@${IONIC_CLI_VERSION}

# Build for production
RUN npm run build -- --configuration=production

# ---------------------------------

In [50]:
# =============================================================================
# 5.7 NGINX CONFIGURATION - FRONTEND
# =============================================================================

nginx_conf = '''# ============================================================================
# LungLife Frontend - Nginx Configuration
# Purpose: Serve Ionic/Angular SPA with API proxy
# ============================================================================

server {
    listen 80;
    listen [::]:80;
    server_name localhost;
    
    # Root directory
    root /usr/share/nginx/html;
    index index.html;
    
    # Gzip compression
    gzip on;
    gzip_vary on;
    gzip_min_length 1024;
    gzip_proxied expired no-cache no-store private auth;
    gzip_types
        text/plain
        text/css
        text/xml
        text/javascript
        application/javascript
        application/xml+rss
        application/json
        image/svg+xml;
    
    # Security headers
    add_header X-Frame-Options "SAMEORIGIN" always;
    add_header X-Content-Type-Options "nosniff" always;
    add_header X-XSS-Protection "1; mode=block" always;
    add_header Referrer-Policy "strict-origin-when-cross-origin" always;
    add_header Content-Security-Policy "default-src 'self'; script-src 'self' 'unsafe-inline' 'unsafe-eval'; style-src 'self' 'unsafe-inline'; img-src 'self' data: https:; font-src 'self' data:; connect-src 'self' http://localhost:* https://*;" always;
    
    # Cache static assets
    location ~* \\.(jpg|jpeg|png|gif|ico|svg|woff|woff2|ttf|eot)$ {
        expires 1y;
        add_header Cache-Control "public, immutable";
    }
    
    # Cache JavaScript and CSS with version hashing
    location ~* \\.(js|css)$ {
        expires 1y;
        add_header Cache-Control "public, immutable";
    }
    
    # API Proxy to Backend
    location /api/ {
        proxy_pass http://backend:3000/api/;
        proxy_http_version 1.1;
        proxy_set_header Upgrade $http_upgrade;
        proxy_set_header Connection 'upgrade';
        proxy_set_header Host $host;
        proxy_set_header X-Real-IP $remote_addr;
        proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
        proxy_set_header X-Forwarded-Proto $scheme;
        proxy_cache_bypass $http_upgrade;
        
        # Timeouts
        proxy_connect_timeout 30s;
        proxy_send_timeout 60s;
        proxy_read_timeout 60s;
    }
    
    # SPA fallback - Angular routing
    location / {
        try_files $uri $uri/ /index.html;
        
        # No cache for index.html
        location = /index.html {
            add_header Cache-Control "no-cache, no-store, must-revalidate";
            add_header Pragma "no-cache";
            add_header Expires "0";
        }
    }
    
    # Health check endpoint
    location /health {
        access_log off;
        return 200 "healthy\\n";
        add_header Content-Type text/plain;
    }
    
    # Deny access to hidden files
    location ~ /\\. {
        deny all;
        access_log off;
        log_not_found off;
    }
}
'''

print("=" * 80)
print("NGINX CONFIGURATION - FRONTEND")
print("=" * 80)
print(nginx_conf)

# Guardar archivo
nginx_conf_path = DEPLOYMENT_DIR / 'frontend' / 'nginx.conf'
with open(nginx_conf_path, 'w', encoding='utf-8') as f:
    f.write(nginx_conf)
print(f"\n[OK] nginx.conf guardado en: {nginx_conf_path}")

NGINX CONFIGURATION - FRONTEND
# ============================================================================
# LungLife Frontend - Nginx Configuration
# Purpose: Serve Ionic/Angular SPA with API proxy
# ============================================================================

server {
    listen 80;
    listen [::]:80;
    server_name localhost;

    # Root directory
    root /usr/share/nginx/html;
    index index.html;

    # Gzip compression
    gzip on;
    gzip_vary on;
    gzip_min_length 1024;
    gzip_proxied expired no-cache no-store private auth;
    gzip_types
        text/plain
        text/css
        text/xml
        text/javascript
        application/javascript
        application/xml+rss
        application/json
        image/svg+xml;

    # Security headers
    add_header X-Frame-Options "SAMEORIGIN" always;
    add_header X-Content-Type-Options "nosniff" always;
    add_header X-XSS-Protection "1; mode=block" always;
    add_header Referrer-Policy "strict-origin-

In [51]:
# =============================================================================
# 5.8 ENVIRONMENT CONFIGURATION (.env.example)
# =============================================================================

env_example = '''# ============================================================================
# LungLife MVP - Environment Variables
# Copy this file to .env and update values for your environment
# ============================================================================

# =============================================================================
# GENERAL
# =============================================================================
ENVIRONMENT=production
LOG_LEVEL=INFO

# =============================================================================
# DATABASE (PostgreSQL)
# =============================================================================
POSTGRES_HOST=postgres
POSTGRES_PORT=5432
POSTGRES_DB=lunglife
POSTGRES_USER=lunglife_user
POSTGRES_PASSWORD=CHANGE_THIS_SECURE_PASSWORD_123!

# =============================================================================
# REDIS
# =============================================================================
REDIS_HOST=redis
REDIS_PORT=6379
REDIS_PASSWORD=

# =============================================================================
# JWT AUTHENTICATION
# =============================================================================
JWT_SECRET=CHANGE_THIS_TO_A_VERY_LONG_RANDOM_STRING_AT_LEAST_64_CHARS
JWT_EXPIRES_IN=24h
JWT_REFRESH_EXPIRES_IN=7d

# =============================================================================
# ML SERVICE
# =============================================================================
ML_SERVICE_URL=http://ml-service:8000
ML_SERVICE_PORT=8000
ML_MAX_WORKERS=4
MODEL_VERSION=1.0.0
MODEL_PATH=/app/models

# =============================================================================
# BACKEND
# =============================================================================
BACKEND_PORT=3000
NODE_ENV=production

# =============================================================================
# FRONTEND
# =============================================================================
FRONTEND_PORT=8100
API_URL=http://localhost:3000/api

# =============================================================================
# SECURITY
# =============================================================================
CORS_ORIGINS=http://localhost:8100,http://localhost:4200
RATE_LIMIT_WINDOW_MS=900000
RATE_LIMIT_MAX_REQUESTS=100

# =============================================================================
# MONITORING (Optional)
# =============================================================================
# SENTRY_DSN=
# NEW_RELIC_LICENSE_KEY=
# DATADOG_API_KEY=
'''

print("=" * 80)
print("ENVIRONMENT CONFIGURATION (.env.example)")
print("=" * 80)
print(env_example)

# Guardar archivo
env_path = DEPLOYMENT_DIR / '.env.example'
with open(env_path, 'w', encoding='utf-8') as f:
    f.write(env_example)
print(f"\n[OK] .env.example guardado en: {env_path}")

ENVIRONMENT CONFIGURATION (.env.example)
# ============================================================================
# LungLife MVP - Environment Variables
# Copy this file to .env and update values for your environment
# ============================================================================

# =============================================================================
# GENERAL
# =============================================================================
ENVIRONMENT=production
LOG_LEVEL=INFO

# =============================================================================
# DATABASE (PostgreSQL)
# =============================================================================
POSTGRES_HOST=postgres
POSTGRES_PORT=5432
POSTGRES_DB=lunglife
POSTGRES_USER=lunglife_user
POSTGRES_PASSWORD=CHANGE_THIS_SECURE_PASSWORD_123!

# =============================================================================
# REDIS
# =======================================================================

---

## 6. Documentación OpenAPI/Swagger

Especificación OpenAPI 3.0 para la documentación interactiva de la API del servicio ML.

In [52]:
# =============================================================================
# 6.1 OPENAPI SPECIFICATION (openapi.yaml)
# =============================================================================

openapi_spec = '''# ============================================================================
# LungLife ML Service - OpenAPI 3.0 Specification
# Version: 1.0.0
# ============================================================================
openapi: 3.0.3

info:
  title: LungLife ML Service API
  description: |
    API REST para el servicio de Machine Learning del MVP LungLife.
    Proporciona predicciones de riesgo de cáncer pulmonar basadas en
    factores clínicos y demográficos del paciente.
    
    ## Características
    - Predicción de riesgo con interpretabilidad (SHAP)
    - Clasificación por niveles de riesgo clínico
    - Soporte para predicciones individuales y por lotes
    - Health checks y métricas de rendimiento
    
    ## Autenticación
    Todas las rutas de predicción requieren autenticación Bearer JWT.
  version: 1.0.0
  contact:
    name: LungLife Team
    email: soporte@lunglife.cl
  license:
    name: Proprietary
    url: https://lunglife.cl/license

servers:
  - url: http://localhost:8000
    description: Development server
  - url: https://api.lunglife.cl/ml
    description: Production server

tags:
  - name: Health
    description: Health check endpoints
  - name: Predictions
    description: ML prediction endpoints
  - name: Model
    description: Model information endpoints

paths:
  /health:
    get:
      tags:
        - Health
      summary: Health Check
      description: Verifica el estado del servicio y sus dependencias
      operationId: healthCheck
      responses:
        '200':
          description: Servicio saludable
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/HealthResponse'
              example:
                status: healthy
                timestamp: "2024-01-15T10:30:00Z"
                version: "1.0.0"
                model_loaded: true
                uptime_seconds: 3600.5

  /api/v1/predict:
    post:
      tags:
        - Predictions
      summary: Predicción Individual
      description: |
        Realiza una predicción de riesgo de cáncer pulmonar para un paciente.
        Retorna probabilidad, nivel de riesgo y top 5 factores contribuyentes.
      operationId: predictSingle
      security:
        - bearerAuth: []
      requestBody:
        required: true
        content:
          application/json:
            schema:
              $ref: '#/components/schemas/PatientDataRequest'
            example:
              age: 65
              gender: "M"
              smoking: 2
              yellow_fingers: 1
              anxiety: 1
              peer_pressure: 0
              chronic_disease: 1
              fatigue: 2
              allergy: 0
              wheezing: 2
              alcohol_consuming: 1
              coughing: 2
              shortness_of_breath: 2
              swallowing_difficulty: 1
              chest_pain: 2
      responses:
        '200':
          description: Predicción exitosa
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/PredictionResponse'
              example:
                prediction: 1
                probability: 0.847
                risk_level: "HIGH"
                confidence: 0.923
                model_version: "1.0.0"
                top_features:
                  - feature: "smoking"
                    contribution: 0.234
                    direction: "positive"
                  - feature: "age"
                    contribution: 0.187
                    direction: "positive"
                  - feature: "wheezing"
                    contribution: 0.156
                    direction: "positive"
                  - feature: "chest_pain"
                    contribution: 0.134
                    direction: "positive"
                  - feature: "chronic_disease"
                    contribution: 0.098
                    direction: "positive"
                recommendation: "Se recomienda evaluación médica urgente"
                timestamp: "2024-01-15T10:30:00Z"
        '400':
          description: Datos de entrada inválidos
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/ErrorResponse'
        '401':
          description: No autenticado
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/ErrorResponse'
        '500':
          description: Error interno del servidor
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/ErrorResponse'

  /api/v1/predict/batch:
    post:
      tags:
        - Predictions
      summary: Predicción por Lotes
      description: |
        Realiza predicciones para múltiples pacientes en una sola solicitud.
        Máximo 100 pacientes por lote.
      operationId: predictBatch
      security:
        - bearerAuth: []
      requestBody:
        required: true
        content:
          application/json:
            schema:
              $ref: '#/components/schemas/BatchPredictionRequest'
      responses:
        '200':
          description: Predicciones exitosas
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/BatchPredictionResponse'
        '400':
          description: Datos de entrada inválidos
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/ErrorResponse'

  /api/v1/model/info:
    get:
      tags:
        - Model
      summary: Información del Modelo
      description: Retorna metadata del modelo cargado
      operationId: getModelInfo
      security:
        - bearerAuth: []
      responses:
        '200':
          description: Información del modelo
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/ModelInfoResponse'

components:
  securitySchemes:
    bearerAuth:
      type: http
      scheme: bearer
      bearerFormat: JWT
      description: JWT token from backend authentication

  schemas:
    PatientDataRequest:
      type: object
      required:
        - age
        - gender
        - smoking
      properties:
        age:
          type: integer
          minimum: 18
          maximum: 120
          description: Edad del paciente en años
        gender:
          type: string
          enum: ["M", "F"]
          description: Género del paciente
        smoking:
          type: integer
          minimum: 0
          maximum: 2
          description: "Nivel de tabaquismo (0: No, 1: Ex-fumador, 2: Fumador activo)"
        yellow_fingers:
          type: integer
          minimum: 0
          maximum: 2
          default: 0
        anxiety:
          type: integer
          minimum: 0
          maximum: 2
          default: 0
        peer_pressure:
          type: integer
          minimum: 0
          maximum: 2
          default: 0
        chronic_disease:
          type: integer
          minimum: 0
          maximum: 2
          default: 0
        fatigue:
          type: integer
          minimum: 0
          maximum: 2
          default: 0
        allergy:
          type: integer
          minimum: 0
          maximum: 2
          default: 0
        wheezing:
          type: integer
          minimum: 0
          maximum: 2
          default: 0
        alcohol_consuming:
          type: integer
          minimum: 0
          maximum: 2
          default: 0
        coughing:
          type: integer
          minimum: 0
          maximum: 2
          default: 0
        shortness_of_breath:
          type: integer
          minimum: 0
          maximum: 2
          default: 0
        swallowing_difficulty:
          type: integer
          minimum: 0
          maximum: 2
          default: 0
        chest_pain:
          type: integer
          minimum: 0
          maximum: 2
          default: 0

    PredictionResponse:
      type: object
      properties:
        prediction:
          type: integer
          enum: [0, 1]
          description: "Predicción binaria (0: No cáncer, 1: Riesgo de cáncer)"
        probability:
          type: number
          format: float
          minimum: 0
          maximum: 1
          description: Probabilidad de riesgo (0.0 a 1.0)
        risk_level:
          type: string
          enum: ["LOW", "MODERATE", "HIGH", "VERY_HIGH"]
          description: Nivel de riesgo clínico
        confidence:
          type: number
          format: float
          description: Confianza del modelo en la predicción
        model_version:
          type: string
          description: Versión del modelo utilizado
        top_features:
          type: array
          items:
            $ref: '#/components/schemas/FeatureContribution'
          description: Top 5 factores contribuyentes
        recommendation:
          type: string
          description: Recomendación clínica basada en el riesgo
        timestamp:
          type: string
          format: date-time
          description: Timestamp de la predicción

    FeatureContribution:
      type: object
      properties:
        feature:
          type: string
          description: Nombre del factor
        contribution:
          type: number
          format: float
          description: Magnitud de la contribución
        direction:
          type: string
          enum: ["positive", "negative"]
          description: Dirección del impacto en el riesgo

    BatchPredictionRequest:
      type: object
      properties:
        patients:
          type: array
          items:
            $ref: '#/components/schemas/PatientDataRequest'
          maxItems: 100
          description: Lista de pacientes para predicción

    BatchPredictionResponse:
      type: object
      properties:
        predictions:
          type: array
          items:
            $ref: '#/components/schemas/PredictionResponse'
        total_processed:
          type: integer
        processing_time_ms:
          type: number

    HealthResponse:
      type: object
      properties:
        status:
          type: string
          enum: ["healthy", "degraded", "unhealthy"]
        timestamp:
          type: string
          format: date-time
        version:
          type: string
        model_loaded:
          type: boolean
        uptime_seconds:
          type: number

    ModelInfoResponse:
      type: object
      properties:
        model_name:
          type: string
        version:
          type: string
        trained_date:
          type: string
          format: date-time
        features:
          type: array
          items:
            type: string
        metrics:
          type: object
          properties:
            accuracy:
              type: number
            recall:
              type: number
            precision:
              type: number
            f1_score:
              type: number
            auc_roc:
              type: number

    ErrorResponse:
      type: object
      properties:
        error:
          type: string
          description: Tipo de error
        message:
          type: string
          description: Mensaje descriptivo
        detail:
          type: object
          description: Detalles adicionales del error
        timestamp:
          type: string
          format: date-time
'''

print("=" * 80)
print("OPENAPI 3.0 SPECIFICATION")
print("=" * 80)
print(openapi_spec[:3000] + "\n... (contenido completo guardado en archivo)")

# Guardar archivo
openapi_path = DEPLOYMENT_DIR / 'docs' / 'openapi.yaml'
with open(openapi_path, 'w', encoding='utf-8') as f:
    f.write(openapi_spec)
print(f"\n[OK] openapi.yaml guardado en: {openapi_path}")

OPENAPI 3.0 SPECIFICATION
# ============================================================================
# LungLife ML Service - OpenAPI 3.0 Specification
# Version: 1.0.0
# ============================================================================
openapi: 3.0.3

info:
  title: LungLife ML Service API
  description: |
    API REST para el servicio de Machine Learning del MVP LungLife.
    Proporciona predicciones de riesgo de cáncer pulmonar basadas en
    factores clínicos y demográficos del paciente.

    ## Características
    - Predicción de riesgo con interpretabilidad (SHAP)
    - Clasificación por niveles de riesgo clínico
    - Soporte para predicciones individuales y por lotes
    - Health checks y métricas de rendimiento

    ## Autenticación
    Todas las rutas de predicción requieren autenticación Bearer JWT.
  version: 1.0.0
  contact:
    name: LungLife Team
    email: soporte@lunglife.cl
  license:
    name: Proprietary
    url: https://lunglife.cl/license

servers:
 

---

## 7. JSON Schemas para Intercambio de Datos

Esquemas JSON para validación de datos entre Frontend, Backend y ML Service.

In [53]:
# =============================================================================
# 7.1 DIAGRAMA DE FLUJO DE DATOS JSON
# =============================================================================

json_flow_diagram = """
╔══════════════════════════════════════════════════════════════════════════════╗
║               FLUJO DE DATOS JSON - FRONTEND ↔ BACKEND ↔ ML                  ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  ┌─────────────────────────────────────────────────────────────────────────┐ ║
║  │                         FRONTEND (Ionic/Angular)                        │ ║
║  │                                                                         │ ║
║  │  FormData → PredictionRequest (camelCase)                               │ ║
║  │  {                                                                      │ ║
║  │    "patientId": "uuid",                                                 │ ║
║  │    "age": 65,                                                           │ ║
║  │    "gender": "M",                                                       │ ║
║  │    "smoking": 2,                                                        │ ║
║  │    "yellowFingers": 1,  ← camelCase                                     │ ║
║  │    "chronicDisease": 1,                                                 │ ║
║  │    ...                                                                  │ ║
║  │  }                                                                      │ ║
║  └────────────────────────────────┬────────────────────────────────────────┘ ║
║                                   │                                          ║
║                                   ▼ POST /api/patients/{id}/predict          ║
║                                                                              ║
║  ┌─────────────────────────────────────────────────────────────────────────┐ ║
║  │                          BACKEND (Node.js)                              │ ║
║  │                                                                         │ ║
║  │  ┌─────────────────┐    ┌─────────────────┐    ┌─────────────────┐     │ ║
║  │  │ Recibe Request  │ → │ Transforma a    │ → │ Envía a ML      │     │ ║
║  │  │ (camelCase)     │    │ snake_case      │    │ Service         │     │ ║
║  │  └─────────────────┘    └─────────────────┘    └─────────────────┘     │ ║
║  │                                                                         │ ║
║  │  Transformación: yellowFingers → yellow_fingers                         │ ║
║  │                                                                         │ ║
║  └────────────────────────────────┬────────────────────────────────────────┘ ║
║                                   │                                          ║
║                                   ▼ POST http://ml-service:8000/api/v1/predict║
║                                                                              ║
║  ┌─────────────────────────────────────────────────────────────────────────┐ ║
║  │                         ML SERVICE (FastAPI)                            │ ║
║  │                                                                         │ ║
║  │  PatientDataRequest (snake_case)                                        │ ║
║  │  {                                                                      │ ║
║  │    "age": 65,                                                           │ ║
║  │    "gender": "M",                                                       │ ║
║  │    "smoking": 2,                                                        │ ║
║  │    "yellow_fingers": 1,  ← snake_case                                   │ ║
║  │    "chronic_disease": 1,                                                │ ║
║  │    ...                                                                  │ ║
║  │  }                                                                      │ ║
║  │                                                                         │ ║
║  │  ┌─────────────────┐    ┌─────────────────┐    ┌─────────────────┐     │ ║
║  │  │ Valida Pydantic │ → │ Ejecuta Modelo  │ → │ Calcula SHAP    │     │ ║
║  │  └─────────────────┘    └─────────────────┘    └─────────────────┘     │ ║
║  │                                                                         │ ║
║  │  PredictionResponse (snake_case)                                        │ ║
║  │  {                                                                      │ ║
║  │    "prediction": 1,                                                     │ ║
║  │    "probability": 0.847,                                                │ ║
║  │    "risk_level": "HIGH",                                                │ ║
║  │    "top_features": [...]                                                │ ║
║  │  }                                                                      │ ║
║  └────────────────────────────────┬────────────────────────────────────────┘ ║
║                                   │                                          ║
║                                   ▼ Response                                 ║
║                                                                              ║
║  ┌─────────────────────────────────────────────────────────────────────────┐ ║
║  │                          BACKEND (Node.js)                              │ ║
║  │                                                                         │ ║
║  │  ┌─────────────────┐    ┌─────────────────┐    ┌─────────────────┐     │ ║
║  │  │ Recibe Response │ → │ Guarda en DB    │ → │ Transforma a    │     │ ║
║  │  │ (snake_case)    │    │ (PostgreSQL)    │    │ camelCase       │     │ ║
║  │  └─────────────────┘    └─────────────────┘    └─────────────────┘     │ ║
║  │                                                                         │ ║
║  │  Transformación: risk_level → riskLevel                                 │ ║
║  │                                                                         │ ║
║  └────────────────────────────────┬────────────────────────────────────────┘ ║
║                                   │                                          ║
║                                   ▼ Response                                 ║
║                                                                              ║
║  ┌─────────────────────────────────────────────────────────────────────────┐ ║
║  │                         FRONTEND (Ionic/Angular)                        │ ║
║  │                                                                         │ ║
║  │  PredictionResponse (camelCase)                                         │ ║
║  │  {                                                                      │ ║
║  │    "prediction": 1,                                                     │ ║
║  │    "probability": 0.847,                                                │ ║
║  │    "riskLevel": "HIGH",  ← camelCase                                    │ ║
║  │    "topFeatures": [                                                     │ ║
║  │      { "feature": "smoking", "contribution": 0.234 }                    │ ║
║  │    ]                                                                    │ ║
║  │  }                                                                      │ ║
║  └─────────────────────────────────────────────────────────────────────────┘ ║
║                                                                              ║
╚══════════════════════════════════════════════════════════════════════════════╝
"""

print(json_flow_diagram)

# Guardar diagrama
json_flow_path = DEPLOYMENT_DIR / 'docs' / 'json_flow_diagram.txt'
with open(json_flow_path, 'w', encoding='utf-8') as f:
    f.write(json_flow_diagram)
print(f"\n[OK] Diagrama de flujo JSON guardado en: {json_flow_path}")


╔══════════════════════════════════════════════════════════════════════════════╗
║               FLUJO DE DATOS JSON - FRONTEND ↔ BACKEND ↔ ML                  ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  ┌─────────────────────────────────────────────────────────────────────────┐ ║
║  │                         FRONTEND (Ionic/Angular)                        │ ║
║  │                                                                         │ ║
║  │  FormData → PredictionRequest (camelCase)                               │ ║
║  │  {                                                                      │ ║
║  │    "patientId": "uuid",                                                 │ ║
║  │    "age": 65,                                                           │ ║
║  │    "gender": "M",                                                       │ ║
║  │    "smoking": 2,      

In [54]:
# =============================================================================
# 7.2 JSON SCHEMAS - FRONTEND REQUEST/RESPONSE
# =============================================================================
import json

# Schema para Request del Frontend
frontend_request_schema = {
    "$schema": "http://json-schema.org/draft-07/schema#",
    "$id": "lunglife-frontend-prediction-request",
    "title": "Frontend Prediction Request",
    "description": "Schema para solicitud de predicción desde el Frontend (camelCase)",
    "type": "object",
    "required": ["patientId", "age", "gender", "smoking"],
    "properties": {
        "patientId": {
            "type": "string",
            "format": "uuid",
            "description": "UUID del paciente"
        },
        "age": {
            "type": "integer",
            "minimum": 18,
            "maximum": 120,
            "description": "Edad del paciente en años"
        },
        "gender": {
            "type": "string",
            "enum": ["M", "F"],
            "description": "Género del paciente"
        },
        "smoking": {
            "type": "integer",
            "minimum": 0,
            "maximum": 2,
            "description": "Nivel de tabaquismo (0: No, 1: Ex-fumador, 2: Activo)"
        },
        "yellowFingers": {
            "type": "integer",
            "minimum": 0,
            "maximum": 2,
            "default": 0,
            "description": "Dedos amarillos por nicotina"
        },
        "anxiety": {
            "type": "integer",
            "minimum": 0,
            "maximum": 2,
            "default": 0
        },
        "peerPressure": {
            "type": "integer",
            "minimum": 0,
            "maximum": 2,
            "default": 0
        },
        "chronicDisease": {
            "type": "integer",
            "minimum": 0,
            "maximum": 2,
            "default": 0
        },
        "fatigue": {
            "type": "integer",
            "minimum": 0,
            "maximum": 2,
            "default": 0
        },
        "allergy": {
            "type": "integer",
            "minimum": 0,
            "maximum": 2,
            "default": 0
        },
        "wheezing": {
            "type": "integer",
            "minimum": 0,
            "maximum": 2,
            "default": 0
        },
        "alcoholConsuming": {
            "type": "integer",
            "minimum": 0,
            "maximum": 2,
            "default": 0
        },
        "coughing": {
            "type": "integer",
            "minimum": 0,
            "maximum": 2,
            "default": 0
        },
        "shortnessOfBreath": {
            "type": "integer",
            "minimum": 0,
            "maximum": 2,
            "default": 0
        },
        "swallowingDifficulty": {
            "type": "integer",
            "minimum": 0,
            "maximum": 2,
            "default": 0
        },
        "chestPain": {
            "type": "integer",
            "minimum": 0,
            "maximum": 2,
            "default": 0
        }
    },
    "additionalProperties": False
}

# Schema para Response del Frontend
frontend_response_schema = {
    "$schema": "http://json-schema.org/draft-07/schema#",
    "$id": "lunglife-frontend-prediction-response",
    "title": "Frontend Prediction Response",
    "description": "Schema para respuesta de predicción hacia el Frontend (camelCase)",
    "type": "object",
    "required": ["prediction", "probability", "riskLevel", "modelVersion"],
    "properties": {
        "prediction": {
            "type": "integer",
            "enum": [0, 1],
            "description": "Predicción binaria (0: Sin riesgo, 1: Con riesgo)"
        },
        "probability": {
            "type": "number",
            "minimum": 0,
            "maximum": 1,
            "description": "Probabilidad de riesgo"
        },
        "riskLevel": {
            "type": "string",
            "enum": ["LOW", "MODERATE", "HIGH", "VERY_HIGH"],
            "description": "Nivel de riesgo clínico"
        },
        "confidence": {
            "type": "number",
            "minimum": 0,
            "maximum": 1,
            "description": "Confianza del modelo"
        },
        "modelVersion": {
            "type": "string",
            "description": "Versión del modelo ML"
        },
        "topFeatures": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "feature": {"type": "string"},
                    "featureLabel": {"type": "string"},
                    "contribution": {"type": "number"},
                    "direction": {"type": "string", "enum": ["positive", "negative"]}
                }
            },
            "maxItems": 5,
            "description": "Top 5 factores contribuyentes"
        },
        "recommendation": {
            "type": "string",
            "description": "Recomendación clínica"
        },
        "timestamp": {
            "type": "string",
            "format": "date-time",
            "description": "Timestamp de la predicción"
        }
    }
}

print("=" * 80)
print("JSON SCHEMA - FRONTEND REQUEST (camelCase)")
print("=" * 80)
print(json.dumps(frontend_request_schema, indent=2))

print("\n" + "=" * 80)
print("JSON SCHEMA - FRONTEND RESPONSE (camelCase)")
print("=" * 80)
print(json.dumps(frontend_response_schema, indent=2))

# Guardar schemas
schemas_dir = DEPLOYMENT_DIR / 'schemas' / 'json'
schemas_dir.mkdir(parents=True, exist_ok=True)

with open(schemas_dir / 'frontend_request.schema.json', 'w') as f:
    json.dump(frontend_request_schema, f, indent=2)
    
with open(schemas_dir / 'frontend_response.schema.json', 'w') as f:
    json.dump(frontend_response_schema, f, indent=2)

print(f"\n[OK] Schemas guardados en: {schemas_dir}")

JSON SCHEMA - FRONTEND REQUEST (camelCase)
{
  "$schema": "http://json-schema.org/draft-07/schema#",
  "$id": "lunglife-frontend-prediction-request",
  "title": "Frontend Prediction Request",
  "description": "Schema para solicitud de predicci\u00f3n desde el Frontend (camelCase)",
  "type": "object",
  "required": [
    "patientId",
    "age",
    "gender",
    "smoking"
  ],
  "properties": {
    "patientId": {
      "type": "string",
      "format": "uuid",
      "description": "UUID del paciente"
    },
    "age": {
      "type": "integer",
      "minimum": 18,
      "maximum": 120,
      "description": "Edad del paciente en a\u00f1os"
    },
    "gender": {
      "type": "string",
      "enum": [
        "M",
        "F"
      ],
      "description": "G\u00e9nero del paciente"
    },
    "smoking": {
      "type": "integer",
      "minimum": 0,
      "maximum": 2,
      "description": "Nivel de tabaquismo (0: No, 1: Ex-fumador, 2: Activo)"
    },
    "yellowFingers": {
      "type

In [55]:
# =============================================================================
# 7.3 TYPESCRIPT INTERFACES (Frontend Angular)
# =============================================================================

typescript_interfaces = '''// ============================================================================
// LungLife Frontend - TypeScript Interfaces
// Interfaces para comunicación con Backend API
// ============================================================================

/**
 * Enum para niveles de riesgo clínico
 */
export enum RiskLevel {
  LOW = 'LOW',
  MODERATE = 'MODERATE',
  HIGH = 'HIGH',
  VERY_HIGH = 'VERY_HIGH'
}

/**
 * Enum para género del paciente
 */
export enum Gender {
  MALE = 'M',
  FEMALE = 'F'
}

/**
 * Interface para datos del paciente en formulario
 */
export interface PatientFormData {
  patientId: string;
  age: number;
  gender: Gender;
  smoking: number;
  yellowFingers?: number;
  anxiety?: number;
  peerPressure?: number;
  chronicDisease?: number;
  fatigue?: number;
  allergy?: number;
  wheezing?: number;
  alcoholConsuming?: number;
  coughing?: number;
  shortnessOfBreath?: number;
  swallowingDifficulty?: number;
  chestPain?: number;
}

/**
 * Interface para solicitud de predicción al Backend
 */
export interface PredictionRequest extends PatientFormData {
  requestedBy?: string;  // ID del médico
  sessionId?: string;    // ID de sesión
}

/**
 * Interface para contribución de factor
 */
export interface FeatureContribution {
  feature: string;
  featureLabel: string;
  contribution: number;
  direction: 'positive' | 'negative';
}

/**
 * Interface para respuesta de predicción del Backend
 */
export interface PredictionResponse {
  // Resultados del modelo
  prediction: 0 | 1;
  probability: number;
  riskLevel: RiskLevel;
  confidence: number;
  
  // Metadata del modelo
  modelVersion: string;
  
  // Interpretabilidad (SHAP)
  topFeatures: FeatureContribution[];
  
  // Recomendación
  recommendation: string;
  
  // Timestamps
  timestamp: string;
  predictionId?: string;
}

/**
 * Interface para respuesta de error
 */
export interface ApiErrorResponse {
  error: string;
  message: string;
  statusCode: number;
  timestamp: string;
  path?: string;
}

/**
 * Interface para historial de predicciones
 */
export interface PredictionHistory {
  id: string;
  patientId: string;
  prediction: number;
  probability: number;
  riskLevel: RiskLevel;
  createdAt: string;
  createdBy: string;
}

/**
 * Type guard para verificar si es una respuesta de error
 */
export function isApiError(response: unknown): response is ApiErrorResponse {
  return (
    typeof response === 'object' &&
    response !== null &&
    'error' in response &&
    'message' in response
  );
}

/**
 * Mapeo de nombres de features a labels en español
 */
export const FEATURE_LABELS: Record<string, string> = {
  smoking: 'Tabaquismo',
  age: 'Edad',
  yellowFingers: 'Dedos amarillos',
  anxiety: 'Ansiedad',
  peerPressure: 'Presión social',
  chronicDisease: 'Enfermedad crónica',
  fatigue: 'Fatiga',
  allergy: 'Alergia',
  wheezing: 'Sibilancias',
  alcoholConsuming: 'Consumo de alcohol',
  coughing: 'Tos',
  shortnessOfBreath: 'Dificultad respiratoria',
  swallowingDifficulty: 'Dificultad para tragar',
  chestPain: 'Dolor de pecho',
  gender: 'Género'
};

/**
 * Mapeo de niveles de riesgo a colores CSS
 */
export const RISK_LEVEL_COLORS: Record<RiskLevel, string> = {
  [RiskLevel.LOW]: '#4caf50',        // Verde
  [RiskLevel.MODERATE]: '#ff9800',   // Naranja
  [RiskLevel.HIGH]: '#f44336',       // Rojo
  [RiskLevel.VERY_HIGH]: '#9c27b0'   // Púrpura
};

/**
 * Mapeo de niveles de riesgo a labels en español
 */
export const RISK_LEVEL_LABELS: Record<RiskLevel, string> = {
  [RiskLevel.LOW]: 'Riesgo Bajo',
  [RiskLevel.MODERATE]: 'Riesgo Moderado',
  [RiskLevel.HIGH]: 'Riesgo Alto',
  [RiskLevel.VERY_HIGH]: 'Riesgo Muy Alto'
};
'''

print("=" * 80)
print("TYPESCRIPT INTERFACES - FRONTEND ANGULAR")
print("=" * 80)
print(typescript_interfaces)

# Guardar archivo
ts_interfaces_path = DEPLOYMENT_DIR / 'frontend' / 'types' / 'prediction.types.ts'
ts_interfaces_path.parent.mkdir(parents=True, exist_ok=True)
with open(ts_interfaces_path, 'w', encoding='utf-8') as f:
    f.write(typescript_interfaces)
print(f"\n[OK] TypeScript interfaces guardadas en: {ts_interfaces_path}")

TYPESCRIPT INTERFACES - FRONTEND ANGULAR
// ============================================================================
// LungLife Frontend - TypeScript Interfaces
// Interfaces para comunicación con Backend API
// ============================================================================

/**
 * Enum para niveles de riesgo clínico
 */
export enum RiskLevel {
  LOW = 'LOW',
  MODERATE = 'MODERATE',
  HIGH = 'HIGH',
  VERY_HIGH = 'VERY_HIGH'
}

/**
 * Enum para género del paciente
 */
export enum Gender {
  MALE = 'M',
  FEMALE = 'F'
}

/**
 * Interface para datos del paciente en formulario
 */
export interface PatientFormData {
  patientId: string;
  age: number;
  gender: Gender;
  smoking: number;
  yellowFingers?: number;
  anxiety?: number;
  peerPressure?: number;
  chronicDisease?: number;
  fatigue?: number;
  allergy?: number;
  wheezing?: number;
  alcoholConsuming?: number;
  coughing?: number;
  shortnessOfBreath?: number;
  swallowingDifficulty?: number;
  chestPain?: n

---

## 8. Guía de Troubleshooting

Guía de resolución de problemas comunes en el despliegue del MVP LungLife.

In [56]:
# =============================================================================
# 8.1 DIAGRAMA DE DIAGNÓSTICO DE PROBLEMAS
# =============================================================================

troubleshooting_diagram = """
╔══════════════════════════════════════════════════════════════════════════════╗
║                    ÁRBOL DE DIAGNÓSTICO DE PROBLEMAS                         ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║                        ┌─────────────────────┐                               ║
║                        │  ¿El servicio no    │                               ║
║                        │     responde?       │                               ║
║                        └──────────┬──────────┘                               ║
║                                   │                                          ║
║              ┌────────────────────┼────────────────────┐                     ║
║              ▼                    ▼                    ▼                     ║
║    ┌─────────────────┐  ┌─────────────────┐  ┌─────────────────┐            ║
║    │    Frontend     │  │    Backend      │  │   ML Service    │            ║
║    │   (Port 8100)   │  │   (Port 3000)   │  │   (Port 8000)   │            ║
║    └────────┬────────┘  └────────┬────────┘  └────────┬────────┘            ║
║             │                    │                    │                      ║
║             ▼                    ▼                    ▼                      ║
║    ┌─────────────────┐  ┌─────────────────┐  ┌─────────────────┐            ║
║    │ ¿Nginx activo?  │  │ ¿Node running?  │  │ ¿Uvicorn up?    │            ║
║    │                 │  │                 │  │                 │            ║
║    │ docker logs     │  │ docker logs     │  │ docker logs     │            ║
║    │ lunglife_       │  │ lunglife_       │  │ lunglife_       │            ║
║    │ frontend        │  │ backend         │  │ ml_service      │            ║
║    └────────┬────────┘  └────────┬────────┘  └────────┬────────┘            ║
║             │                    │                    │                      ║
║             │                    │                    │                      ║
║  ┌──────────┴──────────┐         │         ┌──────────┴──────────┐          ║
║  │                     │         │         │                     │          ║
║  ▼                     ▼         ▼         ▼                     ▼          ║
║ [Verificar build]  [Config]  [DB/Redis]  [Modelo]           [Deps]         ║
║                                                                              ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  ERRORES COMUNES Y SOLUCIONES                                               ║
║  ─────────────────────────────                                              ║
║                                                                              ║
║  ┌───────────────────────────────────────────────────────────────────────┐  ║
║  │ ERROR: Connection refused (ECONNREFUSED)                              │  ║
║  ├───────────────────────────────────────────────────────────────────────┤  ║
║  │ CAUSA: Servicio no está corriendo o puerto bloqueado                  │  ║
║  │ SOLUCIÓN:                                                             │  ║
║  │   1. docker-compose ps (verificar estado)                             │  ║
║  │   2. docker-compose up -d <servicio>                                  │  ║
║  │   3. netstat -an | grep <puerto>                                      │  ║
║  └───────────────────────────────────────────────────────────────────────┘  ║
║                                                                              ║
║  ┌───────────────────────────────────────────────────────────────────────┐  ║
║  │ ERROR: Model not loaded / Model file not found                        │  ║
║  ├───────────────────────────────────────────────────────────────────────┤  ║
║  │ CAUSA: Archivo del modelo no existe o path incorrecto                 │  ║
║  │ SOLUCIÓN:                                                             │  ║
║  │   1. Verificar MODEL_PATH en .env                                     │  ║
║  │   2. docker exec lunglife_ml_service ls /app/models                   │  ║
║  │   3. Copiar modelo: docker cp model.pkl lunglife_ml_service:/app/     │  ║
║  └───────────────────────────────────────────────────────────────────────┘  ║
║                                                                              ║
║  ┌───────────────────────────────────────────────────────────────────────┐  ║
║  │ ERROR: Database connection failed                                     │  ║
║  ├───────────────────────────────────────────────────────────────────────┤  ║
║  │ CAUSA: PostgreSQL no disponible o credenciales incorrectas            │  ║
║  │ SOLUCIÓN:                                                             │  ║
║  │   1. docker-compose logs postgres                                     │  ║
║  │   2. Verificar DATABASE_URL en .env                                   │  ║
║  │   3. docker exec lunglife_postgres pg_isready                         │  ║
║  └───────────────────────────────────────────────────────────────────────┘  ║
║                                                                              ║
║  ┌───────────────────────────────────────────────────────────────────────┐  ║
║  │ ERROR: 401 Unauthorized                                               │  ║
║  ├───────────────────────────────────────────────────────────────────────┤  ║
║  │ CAUSA: Token JWT expirado o inválido                                  │  ║
║  │ SOLUCIÓN:                                                             │  ║
║  │   1. Verificar header Authorization: Bearer <token>                   │  ║
║  │   2. Verificar JWT_SECRET coincide en todos los servicios             │  ║
║  │   3. Renovar token con endpoint /api/auth/refresh                     │  ║
║  └───────────────────────────────────────────────────────────────────────┘  ║
║                                                                              ║
║  ┌───────────────────────────────────────────────────────────────────────┐  ║
║  │ ERROR: 503 Service Unavailable (Circuit Breaker Open)                 │  ║
║  ├───────────────────────────────────────────────────────────────────────┤  ║
║  │ CAUSA: ML Service ha fallado múltiples veces                          │  ║
║  │ SOLUCIÓN:                                                             │  ║
║  │   1. docker-compose logs ml-service                                   │  ║
║  │   2. docker-compose restart ml-service                                │  ║
║  │   3. Esperar 30s para que circuit breaker se cierre                   │  ║
║  └───────────────────────────────────────────────────────────────────────┘  ║
║                                                                              ║
╚══════════════════════════════════════════════════════════════════════════════╝
"""

print(troubleshooting_diagram)

# Guardar diagrama
troubleshooting_path = DEPLOYMENT_DIR / 'docs' / 'troubleshooting_diagram.txt'
with open(troubleshooting_path, 'w', encoding='utf-8') as f:
    f.write(troubleshooting_diagram)
print(f"\n[OK] Diagrama de troubleshooting guardado en: {troubleshooting_path}")


╔══════════════════════════════════════════════════════════════════════════════╗
║                    ÁRBOL DE DIAGNÓSTICO DE PROBLEMAS                         ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║                        ┌─────────────────────┐                               ║
║                        │  ¿El servicio no    │                               ║
║                        │     responde?       │                               ║
║                        └──────────┬──────────┘                               ║
║                                   │                                          ║
║              ┌────────────────────┼────────────────────┐                     ║
║              ▼                    ▼                    ▼                     ║
║    ┌─────────────────┐  ┌─────────────────┐  ┌─────────────────┐            ║
║    │    Frontend     │  │ 

In [57]:
# =============================================================================
# 8.2 GUÍA DE TROUBLESHOOTING COMPLETA
# =============================================================================

troubleshooting_guide = '''# ============================================================================
# LungLife MVP - Guía de Troubleshooting
# Versión: 1.0.0
# ============================================================================

## 1. VERIFICACIÓN DE ESTADO DE SERVICIOS

### 1.1 Verificar todos los contenedores
```bash
docker-compose ps
```
Salida esperada:
```
NAME                    STATUS              PORTS
lunglife_frontend       Up (healthy)        0.0.0.0:8100->80/tcp
lunglife_backend        Up (healthy)        0.0.0.0:3000->3000/tcp
lunglife_ml_service     Up (healthy)        0.0.0.0:8000->8000/tcp
lunglife_postgres       Up (healthy)        0.0.0.0:5432->5432/tcp
lunglife_redis          Up (healthy)        0.0.0.0:6379->6379/tcp
```

### 1.2 Health Checks individuales
```bash
# Frontend
curl http://localhost:8100/health

# Backend
curl http://localhost:3000/api/health

# ML Service
curl http://localhost:8000/health

# PostgreSQL
docker exec lunglife_postgres pg_isready -U lunglife_user

# Redis
docker exec lunglife_redis redis-cli ping
```

## 2. COMANDOS DE DIAGNÓSTICO

### 2.1 Ver logs en tiempo real
```bash
# Todos los servicios
docker-compose logs -f

# Servicio específico
docker-compose logs -f ml-service
docker-compose logs -f backend
docker-compose logs -f frontend
```

### 2.2 Inspeccionar contenedor
```bash
docker inspect lunglife_ml_service
docker exec -it lunglife_ml_service /bin/sh
```

### 2.3 Verificar uso de recursos
```bash
docker stats
```

### 2.4 Verificar red Docker
```bash
docker network inspect lunglife-network
```

## 3. PROBLEMAS DE CONEXIÓN

### 3.1 Backend no puede conectar a ML Service
**Síntoma:** Error "ECONNREFUSED" o timeout
**Diagnóstico:**
```bash
# Desde dentro del backend
docker exec lunglife_backend curl http://ml-service:8000/health
```
**Soluciones:**
1. Verificar que ml-service esté corriendo
2. Verificar que estén en la misma red Docker
3. Verificar firewall/security groups

### 3.2 Backend no puede conectar a PostgreSQL
**Síntoma:** "Connection refused" o "FATAL: password authentication failed"
**Diagnóstico:**
```bash
docker exec lunglife_backend nc -zv postgres 5432
```
**Soluciones:**
1. Verificar credenciales en .env
2. Esperar a que PostgreSQL esté completamente iniciado
3. Verificar healthcheck de postgres

## 4. PROBLEMAS DE ML SERVICE

### 4.1 Modelo no carga
**Síntoma:** "Model not found" o "Failed to load model"
**Diagnóstico:**
```bash
docker exec lunglife_ml_service ls -la /app/models
docker exec lunglife_ml_service cat /app/logs/ml_service.log
```
**Soluciones:**
1. Verificar que el volumen ml-models esté montado
2. Verificar MODEL_PATH en variables de entorno
3. Verificar permisos del archivo del modelo

### 4.2 Predicciones lentas o timeout
**Síntoma:** Requests tardan > 10s o fallan por timeout
**Diagnóstico:**
```bash
docker exec lunglife_ml_service top
docker stats lunglife_ml_service
```
**Soluciones:**
1. Aumentar memory limit del contenedor
2. Reducir ML_MAX_WORKERS si hay swap excesivo
3. Verificar que el modelo no sea demasiado grande

### 4.3 Error de SHAP/Interpretabilidad
**Síntoma:** "SHAP explainer not available"
**Diagnóstico:**
```bash
docker exec lunglife_ml_service python -c "import shap; print(shap.__version__)"
```
**Soluciones:**
1. Verificar que shap esté instalado en requirements.txt
2. El modelo debe ser compatible con TreeExplainer
3. Deshabilitar SHAP temporalmente con flag DISABLE_SHAP=true

## 5. PROBLEMAS DE FRONTEND

### 5.1 Página en blanco
**Síntoma:** Browser muestra página vacía
**Diagnóstico:**
```bash
docker exec lunglife_frontend cat /usr/share/nginx/html/index.html
docker exec lunglife_frontend nginx -t
```
**Soluciones:**
1. Verificar que el build de Angular fue exitoso
2. Revisar nginx.conf para errores de sintaxis
3. Verificar rutas en angular.json

### 5.2 API calls fallan (CORS)
**Síntoma:** "Access-Control-Allow-Origin" errors
**Soluciones:**
1. Verificar CORS_ORIGINS en backend .env
2. Verificar proxy en nginx.conf
3. Usar /api/ prefix para todas las llamadas

## 6. REINICIO Y RECUPERACIÓN

### 6.1 Reinicio suave
```bash
docker-compose restart <service>
```

### 6.2 Reinicio completo
```bash
docker-compose down
docker-compose up -d
```

### 6.3 Reconstruir desde cero
```bash
docker-compose down -v  # Elimina volúmenes
docker-compose build --no-cache
docker-compose up -d
```

### 6.4 Limpiar recursos Docker
```bash
docker system prune -af
docker volume prune -f
```

## 7. MONITOREO Y ALERTAS

### 7.1 Endpoints de métricas
- ML Service: http://localhost:8000/metrics (Prometheus format)
- Backend: http://localhost:3000/api/metrics

### 7.2 Logs estructurados
Todos los servicios escriben logs en formato JSON a /app/logs/
'''

print("=" * 80)
print("GUÍA DE TROUBLESHOOTING - LUNGLIFE MVP")
print("=" * 80)
print(troubleshooting_guide[:3000] + "\n\n... (contenido completo guardado)")

# Guardar guía
guide_path = DEPLOYMENT_DIR / 'docs' / 'TROUBLESHOOTING.md'
with open(guide_path, 'w', encoding='utf-8') as f:
    f.write(troubleshooting_guide)
print(f"\n[OK] Guía de troubleshooting guardada en: {guide_path}")

GUÍA DE TROUBLESHOOTING - LUNGLIFE MVP
# ============================================================================
# LungLife MVP - Guía de Troubleshooting
# Versión: 1.0.0
# ============================================================================

## 1. VERIFICACIÓN DE ESTADO DE SERVICIOS

### 1.1 Verificar todos los contenedores
```bash
docker-compose ps
```
Salida esperada:
```
NAME                    STATUS              PORTS
lunglife_frontend       Up (healthy)        0.0.0.0:8100->80/tcp
lunglife_backend        Up (healthy)        0.0.0.0:3000->3000/tcp
lunglife_ml_service     Up (healthy)        0.0.0.0:8000->8000/tcp
lunglife_postgres       Up (healthy)        0.0.0.0:5432->5432/tcp
lunglife_redis          Up (healthy)        0.0.0.0:6379->6379/tcp
```

### 1.2 Health Checks individuales
```bash
# Frontend
curl http://localhost:8100/health

# Backend
curl http://localhost:3000/api/health

# ML Service
curl http://localhost:8000/health

# PostgreSQL
docker exec lunglife_p

In [58]:
# =============================================================================
# 8.3 SCRIPTS DE DIAGNÓSTICO AUTOMATIZADO
# =============================================================================

health_check_script = '''#!/usr/bin/env python3
# ============================================================================
# LungLife MVP - Health Check Script
# Ejecutar: python health_check.py
# ============================================================================

import requests
import sys
from datetime import datetime
from typing import Dict, Tuple

# Configuración de servicios
SERVICES = {
    "frontend": {
        "url": "http://localhost:8100/health",
        "port": 8100,
        "description": "Frontend (Nginx)"
    },
    "backend": {
        "url": "http://localhost:3000/api/health",
        "port": 3000,
        "description": "Backend (Node.js)"
    },
    "ml_service": {
        "url": "http://localhost:8000/health",
        "port": 8000,
        "description": "ML Service (FastAPI)"
    }
}

TIMEOUT = 5  # segundos


def check_service(name: str, config: Dict) -> Tuple[bool, str]:
    """
    Verifica el estado de un servicio.
    
    Args:
        name: Nombre del servicio
        config: Configuración del servicio
        
    Returns:
        Tupla (is_healthy, message)
    """
    try:
        response = requests.get(config["url"], timeout=TIMEOUT)
        if response.status_code == 200:
            return True, f"OK (status: {response.status_code})"
        else:
            return False, f"WARN (status: {response.status_code})"
    except requests.ConnectionError:
        return False, "ERROR: Connection refused"
    except requests.Timeout:
        return False, f"ERROR: Timeout ({TIMEOUT}s)"
    except Exception as e:
        return False, f"ERROR: {str(e)}"


def print_report(results: Dict[str, Tuple[bool, str]]) -> None:
    """Imprime reporte de estado."""
    print("\\n" + "=" * 60)
    print(f"LUNGLIFE MVP - HEALTH CHECK REPORT")
    print(f"Timestamp: {datetime.now().isoformat()}")
    print("=" * 60)
    
    all_healthy = True
    
    for name, (is_healthy, message) in results.items():
        status = "✓" if is_healthy else "✗"
        config = SERVICES[name]
        print(f"\\n{status} {config['description']}")
        print(f"  URL: {config['url']}")
        print(f"  Status: {message}")
        
        if not is_healthy:
            all_healthy = False
    
    print("\\n" + "=" * 60)
    if all_healthy:
        print("RESULTADO: Todos los servicios están saludables ✓")
    else:
        print("RESULTADO: Algunos servicios tienen problemas ✗")
    print("=" * 60 + "\\n")


def main():
    """Ejecuta health check de todos los servicios."""
    results = {}
    
    for name, config in SERVICES.items():
        is_healthy, message = check_service(name, config)
        results[name] = (is_healthy, message)
    
    print_report(results)
    
    # Exit code
    all_healthy = all(is_healthy for is_healthy, _ in results.values())
    sys.exit(0 if all_healthy else 1)


if __name__ == "__main__":
    main()
'''

print("=" * 80)
print("SCRIPT DE HEALTH CHECK AUTOMATIZADO")
print("=" * 80)
print(health_check_script)

# Guardar script
scripts_dir = DEPLOYMENT_DIR / 'scripts'
scripts_dir.mkdir(parents=True, exist_ok=True)
health_script_path = scripts_dir / 'health_check.py'
with open(health_script_path, 'w', encoding='utf-8') as f:
    f.write(health_check_script)
print(f"\n[OK] Script de health check guardado en: {health_script_path}")

SCRIPT DE HEALTH CHECK AUTOMATIZADO
#!/usr/bin/env python3
# ============================================================================
# LungLife MVP - Health Check Script
# Ejecutar: python health_check.py
# ============================================================================

import requests
import sys
from datetime import datetime
from typing import Dict, Tuple

# Configuración de servicios
SERVICES = {
    "frontend": {
        "url": "http://localhost:8100/health",
        "port": 8100,
        "description": "Frontend (Nginx)"
    },
    "backend": {
        "url": "http://localhost:3000/api/health",
        "port": 3000,
        "description": "Backend (Node.js)"
    },
    "ml_service": {
        "url": "http://localhost:8000/health",
        "port": 8000,
        "description": "ML Service (FastAPI)"
    }
}

TIMEOUT = 5  # segundos


def check_service(name: str, config: Dict) -> Tuple[bool, str]:
    """
    Verifica el estado de un servicio.

    Args:
        nam

---

## 9. Resumen y Conclusiones

Resumen de la arquitectura de despliegue y próximos pasos para el MVP LungLife.

In [59]:
# =============================================================================
# 9.1 RESUMEN DE ARQUITECTURA DE DESPLIEGUE
# =============================================================================

deployment_summary = """
╔══════════════════════════════════════════════════════════════════════════════╗
║                 RESUMEN DE ARQUITECTURA - LUNGLIFE MVP                       ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  COMPONENTES IMPLEMENTADOS                                                   ║
║  ─────────────────────────                                                   ║
║                                                                              ║
║  ┌─────────────────────────────────────────────────────────────────────────┐ ║
║  │ CAPA              │ TECNOLOGÍA         │ PUERTO   │ ESTADO             │ ║
║  ├───────────────────┼────────────────────┼──────────┼────────────────────┤ ║
║  │ Frontend          │ Ionic/Angular      │ 8100     │ ✓ Configurado      │ ║
║  │ Backend           │ Node.js/Express    │ 3000     │ ✓ Configurado      │ ║
║  │ ML Service        │ FastAPI/Python     │ 8000     │ ✓ Implementado     │ ║
║  │ Database          │ PostgreSQL 15      │ 5432     │ ✓ Schema definido  │ ║
║  │ Cache             │ Redis 7            │ 6379     │ ✓ Configurado      │ ║
║  │ Containerización  │ Docker Compose     │ -        │ ✓ Completo         │ ║
║  └─────────────────────────────────────────────────────────────────────────┘ ║
║                                                                              ║
║  ARCHIVOS GENERADOS                                                          ║
║  ──────────────────                                                          ║
║                                                                              ║
║  deployment/                                                                 ║
║  ├── docker-compose.yml           # Orquestación de servicios                ║
║  ├── .env.example                 # Variables de entorno                     ║
║  ├── ml_service/                                                             ║
║  │   ├── Dockerfile               # Imagen ML Service                        ║
║  │   └── app/                                                                ║
║  │       ├── main.py              # FastAPI app                              ║
║  │       ├── schemas.py           # Modelos Pydantic                         ║
║  │       ├── prediction_router.py # Endpoints de predicción                  ║
║  │       └── ml_service_client.ts # Cliente TypeScript                       ║
║  ├── backend/                                                                ║
║  │   └── Dockerfile               # Imagen Backend                           ║
║  ├── frontend/                                                               ║
║  │   ├── Dockerfile               # Imagen Frontend                          ║
║  │   ├── nginx.conf               # Configuración Nginx                      ║
║  │   └── types/                                                              ║
║  │       └── prediction.types.ts  # TypeScript interfaces                    ║
║  ├── schemas/                                                                ║
║  │   ├── database_schema.sql      # PostgreSQL schema                        ║
║  │   └── json/                                                               ║
║  │       ├── frontend_request.schema.json                                    ║
║  │       └── frontend_response.schema.json                                   ║
║  ├── scripts/                                                                ║
║  │   └── health_check.py          # Script de diagnóstico                    ║
║  └── docs/                                                                   ║
║      ├── openapi.yaml             # Especificación OpenAPI 3.0               ║
║      ├── architecture_diagram.txt                                            ║
║      ├── prediction_flow_diagram.txt                                         ║
║      ├── communication_diagram.txt                                           ║
║      ├── er_diagram.txt                                                      ║
║      ├── docker_architecture.txt                                             ║
║      ├── json_flow_diagram.txt                                               ║
║      ├── troubleshooting_diagram.txt                                         ║
║      └── TROUBLESHOOTING.md                                                  ║
║                                                                              ║
║  PATRONES IMPLEMENTADOS                                                      ║
║  ─────────────────────                                                       ║
║                                                                              ║
║  • Clean Code: Funciones pequeñas, nombres descriptivos, SRP                 ║
║  • Circuit Breaker: Protección contra fallos en cascada                      ║
║  • Retry with Exponential Backoff: Reintentos inteligentes                   ║
║  • Health Checks: Verificación de estado de servicios                        ║
║  • Structured Logging: Logs en formato JSON                                  ║
║  • Multi-stage Docker builds: Imágenes optimizadas                           ║
║  • Non-root containers: Seguridad en contenedores                            ║
║  • CORS y Security Headers: Protección de API                                ║
║                                                                              ║
╚══════════════════════════════════════════════════════════════════════════════╝
"""

print(deployment_summary)

# Guardar resumen
summary_path = DEPLOYMENT_DIR / 'docs' / 'deployment_summary.txt'
with open(summary_path, 'w', encoding='utf-8') as f:
    f.write(deployment_summary)
print(f"\n[OK] Resumen de despliegue guardado en: {summary_path}")


╔══════════════════════════════════════════════════════════════════════════════╗
║                 RESUMEN DE ARQUITECTURA - LUNGLIFE MVP                       ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  COMPONENTES IMPLEMENTADOS                                                   ║
║  ─────────────────────────                                                   ║
║                                                                              ║
║  ┌─────────────────────────────────────────────────────────────────────────┐ ║
║  │ CAPA              │ TECNOLOGÍA         │ PUERTO   │ ESTADO             │ ║
║  ├───────────────────┼────────────────────┼──────────┼────────────────────┤ ║
║  │ Frontend          │ Ionic/Angular      │ 8100     │ ✓ Configurado      │ ║
║  │ Backend           │ Node.js/Express    │ 3000     │ ✓ Configurado      │ ║
║  │ ML Service        │ FastAP

In [60]:
# =============================================================================
# 9.2 CHECKLIST DE DEPLOYMENT
# =============================================================================

deployment_checklist = """
╔══════════════════════════════════════════════════════════════════════════════╗
║                      CHECKLIST DE DEPLOYMENT - MVP                           ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  PRE-DEPLOYMENT                                                              ║
║  ──────────────                                                              ║
║  [ ] Modelo ML entrenado y validado (05_evaluation.ipynb)                    ║
║  [ ] Archivo del modelo exportado (model.pkl / model.joblib)                 ║
║  [ ] Variables de entorno configuradas (.env desde .env.example)             ║
║  [ ] Credenciales de base de datos seguras                                   ║
║  [ ] JWT_SECRET generado (mínimo 64 caracteres aleatorios)                   ║
║  [ ] CORS_ORIGINS configurado para dominios permitidos                       ║
║                                                                              ║
║  CONSTRUCCIÓN                                                                ║
║  ────────────                                                                ║
║  [ ] docker-compose build ejecutado sin errores                              ║
║  [ ] Imágenes Docker creadas correctamente                                   ║
║  [ ] Frontend build de producción (npm run build)                            ║
║  [ ] Backend compilado (npm run build / tsc)                                 ║
║  [ ] Dependencias de producción instaladas                                   ║
║                                                                              ║
║  DATABASE                                                                    ║
║  ────────                                                                    ║
║  [ ] PostgreSQL iniciado y saludable                                         ║
║  [ ] Schema aplicado (database_schema.sql)                                   ║
║  [ ] Usuario de aplicación creado con permisos correctos                     ║
║  [ ] Índices verificados                                                     ║
║  [ ] Triggers de auditoría activos                                           ║
║                                                                              ║
║  ML SERVICE                                                                  ║
║  ──────────                                                                  ║
║  [ ] Modelo cargado correctamente                                            ║
║  [ ] Health check responde OK                                                ║
║  [ ] Endpoint /api/v1/predict funcional                                      ║
║  [ ] SHAP interpretabilidad activa                                           ║
║  [ ] Logs estructurados activos                                              ║
║                                                                              ║
║  BACKEND                                                                     ║
║  ───────                                                                     ║
║  [ ] Conexión a PostgreSQL establecida                                       ║
║  [ ] Conexión a Redis establecida                                            ║
║  [ ] Conexión a ML Service establecida                                       ║
║  [ ] Autenticación JWT funcional                                             ║
║  [ ] Circuit breaker configurado                                             ║
║                                                                              ║
║  FRONTEND                                                                    ║
║  ────────                                                                    ║
║  [ ] Nginx sirviendo archivos estáticos                                      ║
║  [ ] Proxy a backend configurado                                             ║
║  [ ] SPA routing funcional                                                   ║
║  [ ] Assets cacheados correctamente                                          ║
║                                                                              ║
║  SEGURIDAD                                                                   ║
║  ─────────                                                                   ║
║  [ ] HTTPS configurado (producción)                                          ║
║  [ ] Headers de seguridad activos                                            ║
║  [ ] Rate limiting configurado                                               ║
║  [ ] Contenedores ejecutando como non-root                                   ║
║  [ ] Secrets no expuestos en logs                                            ║
║                                                                              ║
║  POST-DEPLOYMENT                                                             ║
║  ───────────────                                                             ║
║  [ ] Health checks pasando                                                   ║
║  [ ] Test de predicción end-to-end exitoso                                   ║
║  [ ] Logs centralizados y accesibles                                         ║
║  [ ] Métricas de rendimiento capturadas                                      ║
║  [ ] Documentación de API accesible (/docs)                                  ║
║                                                                              ║
╚══════════════════════════════════════════════════════════════════════════════╝
"""

print(deployment_checklist)

# Guardar checklist
checklist_path = DEPLOYMENT_DIR / 'docs' / 'DEPLOYMENT_CHECKLIST.txt'
with open(checklist_path, 'w', encoding='utf-8') as f:
    f.write(deployment_checklist)
print(f"\n[OK] Checklist de deployment guardado en: {checklist_path}")


╔══════════════════════════════════════════════════════════════════════════════╗
║                      CHECKLIST DE DEPLOYMENT - MVP                           ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  PRE-DEPLOYMENT                                                              ║
║  ──────────────                                                              ║
║  [ ] Modelo ML entrenado y validado (05_evaluation.ipynb)                    ║
║  [ ] Archivo del modelo exportado (model.pkl / model.joblib)                 ║
║  [ ] Variables de entorno configuradas (.env desde .env.example)             ║
║  [ ] Credenciales de base de datos seguras                                   ║
║  [ ] JWT_SECRET generado (mínimo 64 caracteres aleatorios)                   ║
║  [ ] CORS_ORIGINS configurado para dominios permitidos                       ║
║                          

In [61]:
# =============================================================================
# 9.3 PRÓXIMOS PASOS Y RECOMENDACIONES
# =============================================================================

next_steps = """
╔══════════════════════════════════════════════════════════════════════════════╗
║                    PRÓXIMOS PASOS - ROADMAP MVP                              ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  FASE ACTUAL: MVP v1.0                                                       ║
║  ─────────────────────                                                       ║
║  ✓ Modelo ML entrenado y evaluado                                            ║
║  ✓ API REST implementada (FastAPI)                                           ║
║  ✓ Arquitectura de microservicios definida                                   ║
║  ✓ Containerización Docker completa                                          ║
║  ✓ Documentación OpenAPI                                                     ║
║  ✓ Esquema de base de datos                                                  ║
║                                                                              ║
║  ┌─────────────────────────────────────────────────────────────────────────┐ ║
║  │                         ROADMAP DE EVOLUCIÓN                            │ ║
║  └─────────────────────────────────────────────────────────────────────────┘ ║
║                                                                              ║
║  CORTO PLAZO (1-2 semanas)                                                   ║
║  ─────────────────────────                                                   ║
║  1. Integración del Frontend Ionic existente                                 ║
║     • Conectar formulario de evaluación con API                              ║
║     • Implementar visualización de resultados                                ║
║     • Mostrar gráficos de SHAP contributions                                 ║
║                                                                              ║
║  2. Integración del Backend Node.js existente                                ║
║     • Implementar MLServiceClient en código existente                        ║
║     • Agregar endpoints de predicción                                        ║
║     • Configurar persistencia de predicciones                                ║
║                                                                              ║
║  3. Testing End-to-End                                                       ║
║     • Tests de integración Frontend → Backend → ML                           ║
║     • Tests de carga para ML Service                                         ║
║     • Validación con datos de prueba clínicos                                ║
║                                                                              ║
║  MEDIANO PLAZO (3-4 semanas)                                                 ║
║  ──────────────────────────                                                  ║
║  4. Monitoreo y Observabilidad                                               ║
║     • Prometheus + Grafana para métricas                                     ║
║     • ELK Stack para logs centralizados                                      ║
║     • Alertas para degradación de servicio                                   ║
║                                                                              ║
║  5. CI/CD Pipeline                                                           ║
║     • GitHub Actions para build automático                                   ║
║     • Tests automatizados en cada PR                                         ║
║     • Deployment automatizado a staging                                      ║
║                                                                              ║
║  6. Mejoras de Seguridad                                                     ║
║     • Certificados SSL/TLS                                                   ║
║     • Auditoría de seguridad                                                 ║
║     • Penetration testing                                                    ║
║                                                                              ║
║  LARGO PLAZO (1-2 meses)                                                     ║
║  ─────────────────────                                                       ║
║  7. Escalabilidad                                                            ║
║     • Kubernetes deployment                                                  ║
║     • Auto-scaling de ML Service                                             ║
║     • Load balancing                                                         ║
║                                                                              ║
║  8. Mejoras del Modelo                                                       ║
║     • A/B testing de modelos                                                 ║
║     • Retraining pipeline                                                    ║
║     • Model versioning con MLflow                                            ║
║                                                                              ║
║  9. Funcionalidades Adicionales                                              ║
║     • Historial de predicciones por paciente                                 ║
║     • Dashboard de estadísticas                                              ║
║     • Exportación de reportes PDF                                            ║
║                                                                              ║
║  ┌─────────────────────────────────────────────────────────────────────────┐ ║
║  │                      COMANDOS DE INICIO RÁPIDO                          │ ║
║  └─────────────────────────────────────────────────────────────────────────┘ ║
║                                                                              ║
║  # 1. Copiar modelo entrenado                                                ║
║  cp ../models/best_model.pkl deployment/ml_service/models/                   ║
║                                                                              ║
║  # 2. Configurar variables de entorno                                        ║
║  cp deployment/.env.example deployment/.env                                  ║
║  # Editar .env con valores seguros                                           ║
║                                                                              ║
║  # 3. Construir contenedores                                                 ║
║  cd deployment && docker-compose build                                       ║
║                                                                              ║
║  # 4. Iniciar servicios                                                      ║
║  docker-compose up -d                                                        ║
║                                                                              ║
║  # 5. Verificar estado                                                       ║
║  docker-compose ps                                                           ║
║  python scripts/health_check.py                                              ║
║                                                                              ║
║  # 6. Acceder a servicios                                                    ║
║  Frontend:    http://localhost:8100                                          ║
║  Backend:     http://localhost:3000/api                                      ║
║  ML Service:  http://localhost:8000                                          ║
║  Swagger:     http://localhost:8000/docs                                     ║
║                                                                              ║
╚══════════════════════════════════════════════════════════════════════════════╝
"""

print(next_steps)

# Guardar próximos pasos
next_steps_path = DEPLOYMENT_DIR / 'docs' / 'NEXT_STEPS.txt'
with open(next_steps_path, 'w', encoding='utf-8') as f:
    f.write(next_steps)
print(f"\n[OK] Próximos pasos guardados en: {next_steps_path}")


╔══════════════════════════════════════════════════════════════════════════════╗
║                    PRÓXIMOS PASOS - ROADMAP MVP                              ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  FASE ACTUAL: MVP v1.0                                                       ║
║  ─────────────────────                                                       ║
║  ✓ Modelo ML entrenado y evaluado                                            ║
║  ✓ API REST implementada (FastAPI)                                           ║
║  ✓ Arquitectura de microservicios definida                                   ║
║  ✓ Containerización Docker completa                                          ║
║  ✓ Documentación OpenAPI                                                     ║
║  ✓ Esquema de base de datos                                                  ║
║                          

In [62]:
# =============================================================================
# 9.4 RESUMEN FINAL - ARCHIVOS GENERADOS
# =============================================================================

import os
from pathlib import Path

print("=" * 80)
print("FASE 6: DEPLOYMENT - ARCHIVOS GENERADOS")
print("=" * 80)

# Listar archivos generados
def list_files_recursive(directory: Path, prefix: str = "") -> None:
    """Lista archivos de un directorio recursivamente."""
    if not directory.exists():
        print(f"{prefix}[Directorio no existe]")
        return
    
    items = sorted(directory.iterdir())
    for i, item in enumerate(items):
        is_last = i == len(items) - 1
        connector = "└── " if is_last else "├── "
        
        if item.is_file():
            size = item.stat().st_size
            print(f"{prefix}{connector}{item.name} ({size:,} bytes)")
        else:
            print(f"{prefix}{connector}{item.name}/")
            new_prefix = prefix + ("    " if is_last else "│   ")
            list_files_recursive(item, new_prefix)

print(f"\n{DEPLOYMENT_DIR}/")
list_files_recursive(DEPLOYMENT_DIR)

# Contar archivos y tamaño total
total_files = 0
total_size = 0

for root, dirs, files in os.walk(DEPLOYMENT_DIR):
    for file in files:
        total_files += 1
        total_size += Path(root, file).stat().st_size

print(f"\n{'=' * 80}")
print(f"ESTADÍSTICAS:")
print(f"  • Total archivos generados: {total_files}")
print(f"  • Tamaño total: {total_size:,} bytes ({total_size/1024:.2f} KB)")
print(f"  • Ubicación: {DEPLOYMENT_DIR}")
print("=" * 80)

FASE 6: DEPLOYMENT - ARCHIVOS GENERADOS

..\deployment/
├── .env.example (2,635 bytes)
├── architecture_diagram.txt (8,755 bytes)
├── backend/
│   ├── Dockerfile (2,153 bytes)
│   ├── package.json (2,022 bytes)
│   └── src/
│       └── clients/
│           └── ml-service.client.ts (6,954 bytes)
├── communication_diagram.txt (8,587 bytes)
├── database/
│   ├── schema.sql (11,862 bytes)
│   └── seed.sql (3,346 bytes)
├── database_er_diagram.txt (11,838 bytes)
├── docker/
├── docker-compose.yml (6,487 bytes)
├── docker_architecture_diagram.txt (7,662 bytes)
├── docs/
│   ├── architecture_diagram.txt (7,362 bytes)
│   ├── communication_diagram.txt (6,685 bytes)
│   ├── DEPLOYMENT_CHECKLIST.txt (6,587 bytes)
│   ├── deployment_summary.txt (7,312 bytes)
│   ├── docker_architecture.txt (6,921 bytes)
│   ├── er_diagram.txt (8,811 bytes)
│   ├── json_flow_diagram.txt (10,913 bytes)
│   ├── ml_service_structure.txt (5,285 bytes)
│   ├── NEXT_STEPS.txt (9,611 bytes)
│   ├── openapi.yaml (11,896 b

---

## 10. Conclusiones

### Logros de la Fase 6: Deployment

Esta fase ha establecido la arquitectura completa de despliegue para el MVP de LungLife, siguiendo las mejores prácticas de la industria:

| Aspecto | Implementación |
|---------|----------------|
| **API REST** | FastAPI con endpoints de predicción, health checks y documentación automática |
| **Orquestación** | Docker Compose con 5 servicios interconectados (frontend, backend, ml-service, postgres, redis) |
| **Comunicación** | Cliente HTTP con circuit breaker, retry con backoff exponencial y timeouts |
| **Persistencia** | PostgreSQL con schema completo para predicciones y auditoría |
| **Documentación** | OpenAPI 3.0, JSON Schemas, TypeScript interfaces |
| **Clean Code** | Funciones pequeñas, nombres descriptivos, separación de responsabilidades |
| **Seguridad** | JWT auth, CORS, security headers, contenedores non-root |

### Diagramas Generados

- Arquitectura de despliegue
- Flujo de predicción
- Estructura del ML Service
- Diagrama de comunicación entre servicios
- Diagrama E-R de base de datos
- Arquitectura de contenedores Docker
- Flujo de datos JSON
- Árbol de troubleshooting

### Siguientes Pasos

1. **Inmediato**: Copiar modelo entrenado e iniciar contenedores
2. **Corto plazo**: Integrar frontend y backend existentes
3. **Mediano plazo**: Implementar CI/CD y monitoreo
4. **Largo plazo**: Escalar con Kubernetes y mejorar modelo

---

**Fase 6 completada** ✓

*Notebook generado siguiendo metodología CRISP-DM*  
*LungLife MVP - Predicción de Riesgo de Cáncer Pulmonar*